In [1]:
import zipfile, os
import shutil
import tarfile
import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import os
import json
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from pdfrw import PdfReader
import numpy as np
from nltk import ngrams
import scispacy
import spacy

#  Elsevier

### Abstract Function (ELSEVIER)

In [195]:
def get_from_abstract_Elsevier(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.elsevier.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.elsevier.com/xml/common/schema}abstract"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                        abstract_title = elem.text
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}simple-para":
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [196]:
#Dictionary created by above 'get_from_abstract' function
abs_dict=get_from_abstract_Elsevier('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/099b9fa2-e041-491c-b486-1a3d534e2eee.xml')
abs_dict

{'Background': 'Tolebrutinib is an oral, CNS-penetrant, irreversible inhibitor of Bruton’s tyrosine kinase, an enzyme expressed in B lymphocytes and myeloid cells including microglia, which are major drivers of inflammation in multiple sclerosis. We aimed to determine the dose-response relationship between tolebrutinib and the reduction in new active brain MRI lesions in patients with relapsing multiple sclerosis.',
 'Methods': 'We did a 16-week, phase 2b, randomised, double-blind, placebo-controlled, crossover, dose-finding trial at 40 centres (academic sites, specialty clinics, and general neurology centres) in ten countries in Europe and North America. Eligible participants were adults aged 18–55 years with diagnosed relapsing multiple sclerosis (either relapsing-remitting or relapsing secondary progressive multiple sclerosis), and one or more of the following criteria: at least one relapse within the previous year, at least two relapses within the previous 2 years, or at least one 

### Other Sections Function (ELSEVIER)

In [197]:
def get_sections_from_body_Elsevier(SourceName):
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.elsevier.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.elsevier.com/xml/ja/schema}body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                        section_title = elem.text
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}para":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
                            
        return section_dict

In [5]:
#Dictionary created by above 'get_sections_from_body' function
section_dict=get_sections_from_body_Elsevier('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/099b9fa2-e041-491c-b486-1a3d534e2eee.xml')
section_dict

{'Introduction': 'Multiple sclerosis is an immune-mediated, inflammatory, demyelinating disorder leading to axon loss, neurological morbidity, and accumulating disability. We searched PubMed on May 1, 2021, for clinical studies published in any language, using unrestricted dates and the terms “multiple sclerosis (MS) and Bruton’s tyrosine kinase (BTK)”, and “Bruton’s tyrosine kinase inhibitor”, “chronic lesion”, or “slowly evolving lesion”. Accumulated evidence from these studies shows that chronic neuroinflammation, driven in part by B lymphocytes and activated microglia within the CNS, is a key contributor to disability accumulation in relapsing and progressive multiple sclerosis. Bruton’s tyrosine kinase is a critical signalling element in B lymphocytes and myeloid cells. A phase 2 trial of evobrutinib, a Bruton’s tyrosine kinase inhibitor, has provided proof of concept that targeting this enzyme in patients with multiple sclerosis can lead to improved clinical and MRI outcomes. How

### NCT Function to validate NCT's

In [198]:
def NCT_filter(section_dict):
    NCT_dict={}
    for i in range(0,len(list(section_dict.keys()))):
        #print(i)
        if len(re.findall(r'(NCT[0-9]+)'.format(11),str(list(section_dict.values())[i])))>0:
               NCT_dict[list(section_dict.keys())[i]]=list(section_dict.values())[i]
    return NCT_dict

In [7]:
#Output for the NCT_filter function
NCT_filter(get_sections_from_body_Elsevier('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml'))

{'CAR-T cell therapies': "Among these advances is the development of engineered T-cells with tumor-targeting receptors, the chimeric antigen receptor (CAR)-T cell therapies  Many groups have tested CAR-T cells in clinical trials and have seen varying degrees of success. In 2014, Maude and colleagues studied the efficacy of CAR-T cells targeting CD19 in patients with refractory disease (relapsed acute lymphoblastic leukemia). The authors infused autologous T cells with CD19-directed CAR lentiviral vector (CTL019) and demonstrated the efficacy of the treatment using flow cytometry and PCR assays. CTL019 treatment led to complete remission in 27 patients (90%) including 2 with blinatumomab-refractory disease and 15 who had stem-cell transplantation. The sustained remission was also achieved with an overall survival rate of 78% up to 24\xa0months  In a 2018 phase I/II clinical trial, Enblad et al. studied the safety and efficacy of third-generation CAR-T cells targeting CD19 for fifteen pa

### Function to Extract URL

In [199]:
from urlextract import URLExtract
def URL_Extract(text_to_extract):
    extractor = URLExtract()
    urls = extractor.find_urls(text_to_extract)
    return urls

### To Convert Columns to JSON format

In [428]:
def data_to_json(abstractdataframe,othersectiondataframe,column_name_for_json,XML_file_list,publisher_name):
    json_list=[]
    xml_names_list=[]
    NCT_final_list=[]
    publisher_list=[]
    for xml_name in XML_file_list:
        NCT_total_list=[]
        json_text=""
        xml_df = abstractdataframe.groupby('XML').get_group(xml_name)
        xml_df1 = othersectiondataframe.groupby('XML').get_group(xml_name)        
        #print(dataframename)
        for xml_no in range(0,len(xml_df)):
            if (xml_df['Section Name'].iloc[xml_no]!='NA'):
                json_text = json_text+xml_df[[ 'Section Name', 'NCT Number in Section',
               'Sentence Containing NCT Number', 'Supporting word', 'URL_found']].iloc[xml_no].to_json()+";;;;"
            #To get the NCT List
            if xml_df['NCT Number in Section'].iloc[xml_no] not in NCT_total_list and xml_df['NCT Number in Section'].iloc[xml_no]!='NA':
                NCT_total_list.append(xml_df['NCT Number in Section'].iloc[xml_no])
        
        for xml_no1 in range(0,len(xml_df1)):
            if (xml_df1['Section Name'].iloc[xml_no1]!='NA'):
                json_text = json_text+xml_df1[[ 'Section Name', 'NCT Number in Section',
                'Sentence Containing NCT Number', 'Supporting word', 'URL_found']].iloc[xml_no1].to_json()+";;;;"
            #To get the NCT List
            if xml_df1['NCT Number in Section'].iloc[xml_no1] not in NCT_total_list and xml_df1['NCT Number in Section'].iloc[xml_no1]!='NA':
                NCT_total_list.append(xml_df1['NCT Number in Section'].iloc[xml_no1])
        
        if NCT_total_list==[]:
            NCT_total_list=""
        NCT_final_list.append(NCT_total_list)
        json_list.append(json_text)
        xml_names_list.append(xml_name)
        publisher_list.append(publisher_name)

    json_df = pd.DataFrame(list(zip(xml_names_list,publisher_list,json_list,NCT_final_list)),
                   columns =['XML','Publisher',column_name_for_json,'NCT Full List'])

    return json_df

### Code to Validate NCT from Abstract (Elsevier)

In [434]:
#To Get NCT from Abstract

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
check_list=[]
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.elsevier.com' in shorttext:
                dd=NCT_filter(get_from_abstract_Elsevier(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #count=count+1

                    #To Verify publication NCT
                    for i in range(0,len(list(dd.keys()))):             
                        NCT_refresh=[] 
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                        NCT_refresh=' '.join(NCT_refresh).split()
                        #print(NCT_refresh)
                        if len(NCT_refresh)>0:
                            for search_word in NCT_refresh:
                                if len(search_word)==11:                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        xml_list.append(xml_name)
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        val_list.append('Abstract')
                                        check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                            
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    val_list.append('NA')
                    check_list.append('NA')
            df1_elsevier=pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'])
            

In [435]:
#URL Extraction and Output
df1_elsevier['URL_found']=df1_elsevier['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df1_elsevier.head()

XML Section Name  \
0  014a758a-0be7-4af8-a0f0-1291f198d303.xml           NA   
1  099b9fa2-e041-491c-b486-1a3d534e2eee.xml     Abstract   
2  0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml           NA   
3  0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml           NA   
4  176db363-c021-4d29-87c7-023169200fdf.xml           NA   

  NCT Number in Section                     Sentence Containing NCT Number  \
0                    NA                                                 NA   
1           NCT03889639  a two-step multiple comparison procedure with ...   
2                    NA                                                 NA   
3                    NA                                                 NA   
4                    NA                                                 NA   

                  Supporting word             URL_found  
0                              NA                    []  
1  [ClinicalTrials.gov, register]  [ClinicalTrials.gov]  
2                              NA                    []  
3                              NA                    []  
4                              NA                    []

### Code to Validate NCT from Other Sections (Elsevier)

In [436]:
#To Get NCT from Body
import pandas as pd
import numpy as np

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
section_list=[]
check_list=[]

NCT_final_list={}
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(name)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.elsevier.com' in shorttext:
                dd=NCT_filter(get_sections_from_body_Elsevier(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #To Verify publication NCT
                    NCT_total_list=[]
                    for i in range(0,len(list(dd.keys()))): 
                        #print(len(list(dd.keys())))
                        NCT_refresh=[] 
                        
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                        NCT_refresh=' '.join(NCT_refresh).split() 
                        #NCT_total_list.append(list(set(NCT_refresh)))
                        #print(NCT_refresh)
                        
                        if len(NCT_refresh)>0:
                            
                            for search_word in NCT_refresh:
                                if len(search_word)==11:
                                    #print(search_word)
                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        #print(proximity_text)
                                        #print(xml_name)
                                        xml_list.append(xml_name)
                                        section_list.append(list(dd.keys())[i])
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        
                                        check_words=['register','started','ongoing','underway'
                                                     ,'reported','ClinicalTrials.gov'
                                             ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                        ##val_list.append('Yes')
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                                    #NCT_final_list.append('NA')
                    NCT_dict={xml_name:NCT_total_list}
                    NCT_final_list.update(NCT_dict)
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    section_list.append('NA')
                    check_list.append('NA')
                    NCT_dict={xml_name:'NA'}
                    NCT_final_list.update(NCT_dict)
                    
            df2_elsevier=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'])
#df2=df2.replace('NA',np.nan).dropna(subset=['Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number']).reset_index().drop(columns=['index'])      

In [437]:
#URL Extraction and Output
df2_elsevier['URL_found']=df2_elsevier['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df2_elsevier.head()

XML Section Name  \
0  014a758a-0be7-4af8-a0f0-1291f198d303.xml           NA   
1  099b9fa2-e041-491c-b486-1a3d534e2eee.xml     Outcomes   
2  099b9fa2-e041-491c-b486-1a3d534e2eee.xml     Outcomes   
3  099b9fa2-e041-491c-b486-1a3d534e2eee.xml     Outcomes   
4  099b9fa2-e041-491c-b486-1a3d534e2eee.xml     Outcomes   

  NCT Number in Section                     Sentence Containing NCT Number  \
0                    NA                                                 NA   
1           NCT04410978  and 23 prespecified MRI analyses; this report ...   
2           NCT04410991  23 prespecified MRI analyses; this report focu...   
3           NCT04411641  focuses on two exploratory endpoints that are ...   
4           NCT04458051  on two exploratory endpoints that are most per...   

  Supporting word URL_found  
0              NA        []  
1       [ongoing]        []  
2       [ongoing]        []  
3       [ongoing]        []  
4       [ongoing]        []

### Final Dataframe (Elsevier)

In [438]:
#To create JSON using data_to_json function
XML_files_unique = df1_elsevier['XML'].unique()
final_df_elsevier = data_to_json(df1_elsevier,df2_elsevier,'NCT_Json',XML_files_unique,'Elsevier')
final_df_elsevier.head()

XML Publisher  \
0  014a758a-0be7-4af8-a0f0-1291f198d303.xml  Elsevier   
1  099b9fa2-e041-491c-b486-1a3d534e2eee.xml  Elsevier   
2  0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml  Elsevier   
3  0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml  Elsevier   
4  176db363-c021-4d29-87c7-023169200fdf.xml  Elsevier   

                                            NCT_Json  \
0                                                      
1  {"Section Name":"Abstract","NCT Number in Sect...   
2                                                      
3                                                      
4                                                      

                                       NCT Full List  
0                                                     
1  [NCT03889639, NCT04410978, NCT04410991, NCT044...  
2                                                     
3                                                     
4

### Extract Title and Journal Title with XML name (Elsevier)

In [608]:
RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd

title_list=[]
journal_list=[]
xml_title_list=[]
actual_title_list=[]
actual_journal_list=[]

for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.elsevier.com' in shorttext:
                #print(SourceFolder)
                tree = etree.parse(SourceFolder)
                roots = tree.getroot()
                for tag in roots.iter("{http://purl.org/dc/terms/}title"):
                    title_list.append(ElementTree.tostring(tag))
                    xml_title_list.append(xml_name)
                
                for tag in roots.iter("{http://prismstandard.org/namespaces/basic/2.0/}publicationName"):
                    journal_list.append(ElementTree.tostring(tag))

#actual Title
for body in title_list:
    root = ElementTree.fromstring(body)
    for elem in root.iter():
        #print(elem.text)
        actual_title_list.append(elem.text)   

#actual Journal
for body in journal_list:
    root = ElementTree.fromstring(body)
    for elem in root.iter():
        #print(elem.text)
        actual_journal_list.append(elem.text)  
        
title_df_elsevier = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
               columns =['XML','Title','Journal Title'])

In [610]:
title_df_elsevier

XML  \
0   014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1   099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
3   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
4   176db363-c021-4d29-87c7-023169200fdf.xml   
5   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
6   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
7   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
8   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
9   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
10  3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
11  3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
12  4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
13  427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
14  42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
15  44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
16  4c527b19-40c1-4b09-812c-7817324e4e66.xml   
17  5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
18  52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
19  5304e59e-0eaf-480e-a51a-04139abd9913.xml   
20  53734523-9df0-460f-972b-3cd7934e1169.xml   
21  5445f1b8-9ded-478a-a835-076bf5758808.xml   
22  54dadc79-920f-4af3-996e-bc9f2cb0f081.xml   
23  5aeb5591-ac1a-4494-b4c8-c9fc5eff2d29.xml   
24  5f14a655-c955-4d3c-a3e6-352fa8935e32.xml   
25  63d3c516-49c0-4b4c-802f-c9b2977bf6e0.xml   
26  6c63b332-015a-4769-bd9f-c39d0f0d5f1e.xml   
27  772b5a08-d556-4fa2-92d5-8c2cc276c0ba.xml   
28  7820b0bb-2cac-454c-8754-c68531076407.xml   
29  7a2ea7cc-d5a6-4928-9ab5-29d366b541ce.xml   
..                                       ...   
31  90e18dfb-84fb-4cc5-af46-15fb6550066c.xml   
32  99b3a468-44a5-4ee8-9f0d-b5e738e6b055.xml   
33  9de73805-a761-4514-b6d1-15c11f7eb20e.xml   
34  a045cf42-352c-4cea-b8b5-96bb8a91e0c6.xml   
35  a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml   
36  aba5f501-aaad-4f67-b8d8-bd01da5d0ec7.xml   
37  aec0be4c-a4e6-435e-b50f-9ce7edd5a8c2.xml   
38  ba2de9ad-2a4c-4c9b-8978-24b07352d887.xml   
39  ba435e02-670f-4b1e-a41e-22f682473f0c.xml   
40  bbc69df3-927a-4878-b029-a368b8834b35.xml   
41  be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
42  c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
43  c1417907-53ee-4d31-a753-cd008516c324.xml   
44  c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
45  c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
46  c512c537-32fb-46b6-adaa-4589799569bc.xml   
47  cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
48  ce1115df-1248-419f-a246-e1f6676556bd.xml   
49  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
50  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
51  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
52  da32c93e-b332-4e10-b07c-6be7035d575b.xml   
53  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
54  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
55  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
56  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
57  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
58  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
59  fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
60  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                Title  \
0   Cancer immunotherapy from biology to nanomedicine   
1   Safety and efficacy of tolebrutinib, an oral b...   
2   Approvals of drugs with uncertain benefit–risk...   
3   Reducing clinical trial risk in multiple scler...   
4   Monoclonal antibodies as immunomodulatory ther...   
5   Mechanisms of action of therapeutic antibodies...   
6   Update in immunosuppressive therapy of myasthe...   
7   Anti-CD20–mediated B-cell depletion in autoimm...   
8   Therapeutic strategies targeting B-cells in mu...   
9   Trial of intrathecal rituximab in progressive ...   
10    B cell targeted therapies in autoimmune disease   
11  Novel Immunotherapeutic Avenues for Rheumatoid...   
12  Second-generation immunotherapeutics in multip...   
13  Clinical translation of immunomodulatory thera...   
14  Ocrelizumab infusion experience in patients wi...   
15  Matching-adjusted indirect treatment compariso...   
16  The COVID-19 pandemic and the use of MS diseas...   
17  Monoclonal Antibodies for Waldenst

In [611]:
df_to_merge_elsevier = pd.merge(title_df_elsevier,final_df_elsevier)
df_to_merge_elsevier

XML  \
0   014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1   099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
3   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
4   176db363-c021-4d29-87c7-023169200fdf.xml   
5   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
6   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
7   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
8   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
9   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
10  3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
11  3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
12  4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
13  427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
14  42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
15  44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
16  4c527b19-40c1-4b09-812c-7817324e4e66.xml   
17  5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
18  52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
19  5304e59e-0eaf-480e-a51a-04139abd9913.xml   
20  53734523-9df0-460f-972b-3cd7934e1169.xml   
21  5445f1b8-9ded-478a-a835-076bf5758808.xml   
22  54dadc79-920f-4af3-996e-bc9f2cb0f081.xml   
23  5aeb5591-ac1a-4494-b4c8-c9fc5eff2d29.xml   
24  5f14a655-c955-4d3c-a3e6-352fa8935e32.xml   
25  63d3c516-49c0-4b4c-802f-c9b2977bf6e0.xml   
26  6c63b332-015a-4769-bd9f-c39d0f0d5f1e.xml   
27  772b5a08-d556-4fa2-92d5-8c2cc276c0ba.xml   
28  7820b0bb-2cac-454c-8754-c68531076407.xml   
29  7a2ea7cc-d5a6-4928-9ab5-29d366b541ce.xml   
..                                       ...   
31  90e18dfb-84fb-4cc5-af46-15fb6550066c.xml   
32  99b3a468-44a5-4ee8-9f0d-b5e738e6b055.xml   
33  9de73805-a761-4514-b6d1-15c11f7eb20e.xml   
34  a045cf42-352c-4cea-b8b5-96bb8a91e0c6.xml   
35  a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml   
36  aba5f501-aaad-4f67-b8d8-bd01da5d0ec7.xml   
37  aec0be4c-a4e6-435e-b50f-9ce7edd5a8c2.xml   
38  ba2de9ad-2a4c-4c9b-8978-24b07352d887.xml   
39  ba435e02-670f-4b1e-a41e-22f682473f0c.xml   
40  bbc69df3-927a-4878-b029-a368b8834b35.xml   
41  be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
42  c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
43  c1417907-53ee-4d31-a753-cd008516c324.xml   
44  c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
45  c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
46  c512c537-32fb-46b6-adaa-4589799569bc.xml   
47  cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
48  ce1115df-1248-419f-a246-e1f6676556bd.xml   
49  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
50  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
51  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
52  da32c93e-b332-4e10-b07c-6be7035d575b.xml   
53  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
54  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
55  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
56  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
57  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
58  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
59  fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
60  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                Title  \
0   Cancer immunotherapy from biology to nanomedicine   
1   Safety and efficacy of tolebrutinib, an oral b...   
2   Approvals of drugs with uncertain benefit–risk...   
3   Reducing clinical trial risk in multiple scler...   
4   Monoclonal antibodies as immunomodulatory ther...   
5   Mechanisms of action of therapeutic antibodies...   
6   Update in immunosuppressive therapy of myasthe...   
7   Anti-CD20–mediated B-cell depletion in autoimm...   
8   Therapeutic strategies targeting B-cells in mu...   
9   Trial of intrathecal rituximab in progressive ...   
10    B cell targeted therapies in autoimmune disease   
11  Novel Immunotherapeutic Avenues for Rheumatoid...   
12  Second-generation immunotherapeutics in multip...   
13  Clinical translation of immunomodulatory thera...   
14  Ocrelizumab infusion experience in patients wi...   
15  Matching-adjusted indirect treatment compariso...   
16  The COVID-19 pandemic and the use of MS diseas...   
17  Monoclonal Antibodies for Waldenst

### Elsevier Other columns extraction

In [231]:
#extraction for elsevier

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
full_body_text = []
body_list = []
NCT_fulltext=[]
tagname1 = "{http://purl.org/dc/terms/}title"
tagname2 = "{http://purl.org/dc/terms/}description"
tagname3 = "{http://www.elsevier.com/xml/common/schema}text"
tagname4 = "{http://www.elsevier.com/xml/common/schema}section-title"
tagname5 = "{http://www.elsevier.com/xml/common/schema}para"
tagname6 = "{http://www.elsevier.com/xml/ja/schema}body"
tagname7 = "{http://www.elsevier.com/xml/common/schema}abstract"

tagname8 = "{http://www.elsevier.com/xml/ja/schema}head"
tagname9 = "{http://www.elsevier.com/xml/ja/schema}article"

tagname1 = "{http://purl.org/dc/terms/}title"
tagname6 = "{http://www.elsevier.com/xml/ja/schema}body"
tagname10 = "{http://prismstandard.org/namespaces/basic/2.0/}doi"
tagname11 = "{http://prismstandard.org/namespaces/basic/2.0/}url"

xml_list_other=[]
title_list = []
body_list = []
doi_list = []
url_list =[]
abstract_list = []
abstract_fulltext = []
abstractcount = 0
bodycount = 0
titlecount = 0
doicount = 0
urlcount = 0
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name) 
        xml_name_other=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.elsevier.com' in shorttext:
                #print(SourceFolder)
                xml_list_other.append(xml_name_other)
                tree = etree.parse(SourceFolder)
                roots = tree.getroot()
                tagname = '{http://www.elsevier.com/xml/ja/schema}body'  
                #get body
                for tag in roots.iter(tagname):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_body_text.append(' '.join(pageText))
                    body_list.append(ElementTree.tostring(tag))
                    bodycount = bodycount + 1
                #get doi    
                for tag in roots.iter(tagname10):
                    doi_list.append(ElementTree.tostring(tag))
                    doicount = doicount + 1
                    
                #get url
                for tag in roots.iter(tagname11):
                    # ElementTree.dump(tag)
                    url_list.append(ElementTree.tostring(tag))
                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    urlcount = urlcount + 1
                
                #get title
                for tag in roots.iter(tagname1):
                    # ElementTree.dump(tag)
                    title_list.append(ElementTree.tostring(tag))
                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    titlecount = titlecount + 1
                    
                #get abstract
                for tag in roots.iter(tagname7):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    abstract_fulltext.append(' '.join(pageText))
                    root_1 = ElementTree.fromstring(tagtext) 
                    abstract_dict = {}  
                    section_title = ""
                    for elem in root_1.iter():
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}title":
                            abstract_dict['Title'] = elem.text
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                            section_title = elem.text
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}simple-para":
                            abstract_dict[section_title] = elem.text
                    abstract_list.append(abstract_dict)
                    abstractcount = abstractcount + 1
                '''
                #Get NCT number
                NCT_list=[]
                for tag in roots.iter("{http://www.elsevier.com/xml/common/schema}para"):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    print(pageText)
                    NCT_list.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(pageText))))
                NCT_list1=' '.join(NCT_list).split()
                if (len(NCT_list1))>0 and (len(NCT_list1))<4:
                    NCT_fulltext.append(list(set(NCT_list1)))
                elif (len(NCT_list1))==0:
                    NCT_fulltext.append('NA')
                else:
                    NCT_fulltext.append('NA')
                '''    
#actual title
count = 0
actual_title_list = []
for body in title_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_title_list.append(elem.text)

#actual doi
count = 0
actual_doi_list = []
for body in doi_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_doi_list.append(elem.text)

        
#actual url
count = 0
actual_url_list = []
for body in url_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_url_list.append(elem.text)
        
#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "{http://www.elsevier.com/xml/common/schema}section-title"
section_paratag = "{http://www.elsevier.com/xml/common/schema}para"

for body in body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
            # print("_____")
            # print(elem.text)
            # print("_____")
            section_title = elem.text
        if elem.tag == section_paratag:
            # print(elem.text)
            para_dict[section_title] = elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1



import pandas as pd
df_e1 = pd.DataFrame(list(zip(xml_list_other,actual_title_list, actual_body_list, actual_doi_list, actual_url_list,full_body_text,abstract_list,abstract_fulltext)),
               columns =['XML','Title', 'Body', 'DOI','URL','Full_body_text','Abstract','Abstract_full'])
# str(my_dict)
def getFullText(text):
    return str(text)

df_e1['full_text'] = df_e1.Body.apply(getFullText)

df_e2 = pd.concat([df_e1,df_e1['Body'].apply(pd.Series)], axis=1)

df_e3 = pd.concat([df_e2,df_e2['Abstract'].apply(pd.Series)], axis=1)

df_elsiver_Ofatumumab = df_e3[['XML','Title','Full_body_text','Abstract_full','DOI','URL']]
df_elsiver_Ofatumumab.shape

(61, 6)

In [232]:
df_elsiver_Ofatumumab

XML  \
0   014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1   099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
3   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
4   176db363-c021-4d29-87c7-023169200fdf.xml   
5   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
6   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
7   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
8   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
9   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
10  3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
11  3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
12  4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
13  427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
14  42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
15  44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
16  4c527b19-40c1-4b09-812c-7817324e4e66.xml   
17  5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
18  52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
19  5304e59e-0eaf-480e-a51a-04139abd9913.xml   
20  53734523-9df0-460f-972b-3cd7934e1169.xml   
21  5445f1b8-9ded-478a-a835-076bf5758808.xml   
22  54dadc79-920f-4af3-996e-bc9f2cb0f081.xml   
23  5aeb5591-ac1a-4494-b4c8-c9fc5eff2d29.xml   
24  5f14a655-c955-4d3c-a3e6-352fa8935e32.xml   
25  63d3c516-49c0-4b4c-802f-c9b2977bf6e0.xml   
26  6c63b332-015a-4769-bd9f-c39d0f0d5f1e.xml   
27  772b5a08-d556-4fa2-92d5-8c2cc276c0ba.xml   
28  7820b0bb-2cac-454c-8754-c68531076407.xml   
29  7a2ea7cc-d5a6-4928-9ab5-29d366b541ce.xml   
..                                       ...   
31  90e18dfb-84fb-4cc5-af46-15fb6550066c.xml   
32  99b3a468-44a5-4ee8-9f0d-b5e738e6b055.xml   
33  9de73805-a761-4514-b6d1-15c11f7eb20e.xml   
34  a045cf42-352c-4cea-b8b5-96bb8a91e0c6.xml   
35  a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml   
36  aba5f501-aaad-4f67-b8d8-bd01da5d0ec7.xml   
37  aec0be4c-a4e6-435e-b50f-9ce7edd5a8c2.xml   
38  ba2de9ad-2a4c-4c9b-8978-24b07352d887.xml   
39  ba435e02-670f-4b1e-a41e-22f682473f0c.xml   
40  bbc69df3-927a-4878-b029-a368b8834b35.xml   
41  be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
42  c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
43  c1417907-53ee-4d31-a753-cd008516c324.xml   
44  c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
45  c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
46  c512c537-32fb-46b6-adaa-4589799569bc.xml   
47  cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
48  ce1115df-1248-419f-a246-e1f6676556bd.xml   
49  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
50  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
51  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
52  da32c93e-b332-4e10-b07c-6be7035d575b.xml   
53  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
54  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
55  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
56  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
57  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
58  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
59  fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
60  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                Title  \
0   Cancer immunotherapy from biology to nanomedicine   
1   Safety and efficacy of tolebrutinib, an oral b...   
2   Approvals of drugs with uncertain benefit–risk...   
3   Reducing clinical trial risk in multiple scler...   
4   Monoclonal antibodies as immunomodulatory ther...   
5   Mechanisms of action of therapeutic antibodies...   
6   Update in immunosuppressive therapy of myasthe...   
7   Anti-CD20–mediated B-cell depletion in autoimm...   
8   Therapeutic strategies targeting B-cells in mu...   
9   Trial of intrathecal rituximab in progressive ...   
10    B cell targeted therapies in autoimmune disease   
11  Novel Immunotherapeutic Avenues for Rheumatoid...   
12  Second-generation immunotherapeutics in multip...   
13  Clinical translation of immunomodulatory thera...   
14  Ocrelizumab infusion experience in patients wi...   
15  Matching-adjusted indirect treatment compariso...   
16  The COVID-19 pandemic and the use of MS diseas...   
17  Monoclonal Antibodies for Waldenst

##  Elsevier Dataframe

In [455]:
Elsevier_data = pd.merge(df_elsiver_Ofatumumab,df_to_merge_elsevier)
Elsevier_data

XML  \
0   014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1   099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
3   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
4   176db363-c021-4d29-87c7-023169200fdf.xml   
5   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
6   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
7   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
8   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
9   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
10  3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
11  3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
12  4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
13  427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
14  42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
15  44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
16  4c527b19-40c1-4b09-812c-7817324e4e66.xml   
17  5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
18  52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
19  5304e59e-0eaf-480e-a51a-04139abd9913.xml   
20  53734523-9df0-460f-972b-3cd7934e1169.xml   
21  5445f1b8-9ded-478a-a835-076bf5758808.xml   
22  54dadc79-920f-4af3-996e-bc9f2cb0f081.xml   
23  5aeb5591-ac1a-4494-b4c8-c9fc5eff2d29.xml   
24  5f14a655-c955-4d3c-a3e6-352fa8935e32.xml   
25  63d3c516-49c0-4b4c-802f-c9b2977bf6e0.xml   
26  6c63b332-015a-4769-bd9f-c39d0f0d5f1e.xml   
27  772b5a08-d556-4fa2-92d5-8c2cc276c0ba.xml   
28  7820b0bb-2cac-454c-8754-c68531076407.xml   
29  7a2ea7cc-d5a6-4928-9ab5-29d366b541ce.xml   
..                                       ...   
31  90e18dfb-84fb-4cc5-af46-15fb6550066c.xml   
32  99b3a468-44a5-4ee8-9f0d-b5e738e6b055.xml   
33  9de73805-a761-4514-b6d1-15c11f7eb20e.xml   
34  a045cf42-352c-4cea-b8b5-96bb8a91e0c6.xml   
35  a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml   
36  aba5f501-aaad-4f67-b8d8-bd01da5d0ec7.xml   
37  aec0be4c-a4e6-435e-b50f-9ce7edd5a8c2.xml   
38  ba2de9ad-2a4c-4c9b-8978-24b07352d887.xml   
39  ba435e02-670f-4b1e-a41e-22f682473f0c.xml   
40  bbc69df3-927a-4878-b029-a368b8834b35.xml   
41  be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
42  c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
43  c1417907-53ee-4d31-a753-cd008516c324.xml   
44  c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
45  c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
46  c512c537-32fb-46b6-adaa-4589799569bc.xml   
47  cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
48  ce1115df-1248-419f-a246-e1f6676556bd.xml   
49  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
50  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
51  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
52  da32c93e-b332-4e10-b07c-6be7035d575b.xml   
53  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
54  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
55  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
56  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
57  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
58  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
59  fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
60  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                Title  \
0   Cancer immunotherapy from biology to nanomedicine   
1   Safety and efficacy of tolebrutinib, an oral b...   
2   Approvals of drugs with uncertain benefit–risk...   
3   Reducing clinical trial risk in multiple scler...   
4   Monoclonal antibodies as immunomodulatory ther...   
5   Mechanisms of action of therapeutic antibodies...   
6   Update in immunosuppressive therapy of myasthe...   
7   Anti-CD20–mediated B-cell depletion in autoimm...   
8   Therapeutic strategies targeting B-cells in mu...   
9   Trial of intrathecal rituximab in progressive ...   
10    B cell targeted therapies in autoimmune disease   
11  Novel Immunotherapeutic Avenues for Rheumatoid...   
12  Second-generation immunotherapeutics in multip...   
13  Clinical translation of immunomodulatory thera...   
14  Ocrelizumab infusion experience in patients wi...   
15  Matching-adjusted indirect treatment compariso...   
16  The COVID-19 pandemic and the use of MS diseas...   
17  Monoclonal Antibodies for Waldenst

In [456]:
writer = pd.ExcelWriter('C:/Users/GOWRICH1/Downloads/HAWK AI/Elsevier_data.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
Elsevier_data.to_excel(writer)
writer.close()

# NLM Extraction

In [5]:
RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'

nlm_body_list = []
full_body_text_nlm = []

for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)       
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                tree = etree.parse(SourceFolder)           
                roots = tree.getroot()  
                print(SourceFolder)
                
                for tag in roots.iter('body'):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_body_text_nlm.append(' '.join(pageText))
                    nlm_body_list.append(ElementTree.tostring(tag))

C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\02be68ce-7530-4882-9a02-d793e6306b43.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\05294995-39f2-48b3-bbae-0a23ffad6c7f.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\068bfff5-8882-4c9b-85ba-6d5dce67aa53.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\0aadf051-3352-4a49-8744-631de0697f0a.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\0bc312b9-a280-4d2c-8ebe-92f48585707f.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\0f2f6b6c-7f6d-4b64-92ca-a16999a09c71.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Process

C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\bc3d0a7a-71b5-4de3-8026-366920053bf0.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\bcce461f-adb2-47dc-bc7a-862b2f5586fc.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\bccfb2dd-30d0-47d3-b6d4-97edcdf4f80c.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\bdae611a-da74-41e0-8e0b-8de922ef0e6c.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\bdc23e24-543f-4bdd-83be-ed7aa5928745.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\c3ada49d-0b6c-4664-9b67-25df5c41e290.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\c723c457-acd1-4a43-be22-4a796692b495.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\c798564e-9105-45e4-b29a-73eb09b2b2fd.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\c7d988be-6699-4a96-8338-497b9984e784.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Processed\ca148833-6462-410e-9898-bc995b01ad4a.xml
C:\Users\GOWRICH1\Downloads\HAWK AI\Process

In [34]:
nlm_body_list = []
full_body_text_nlm = []
SourceFolder='C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/02be68ce-7530-4882-9a02-d793e6306b43.xml'
with open(SourceFolder,'rb') as name:
    data = name.read()
    text = str(data)
    shorttext = str(data)[:100]
    if 'NLM' in shorttext:
        tree = etree.parse(SourceFolder)           
        roots = tree.getroot()  
        print(SourceFolder)

        for tag in roots.iter('body'):
            tagtext = ElementTree.tostring(tag)   
            soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
            pageText = soup.findAll(text=True)
            full_body_text_nlm.append(' '.join(pageText))
            nlm_body_list.append(ElementTree.tostring(tag))
            
            

C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/02be68ce-7530-4882-9a02-d793e6306b43.xml


In [35]:
section_titletag = "title"
section_paratag = "p"
actual_body_list = []
for body in nlm_body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    already_exist_title = set()
    count =0
    existing_text = ""
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
#             print(elem.text)
            section_title = elem.text            
        if elem.tag == section_paratag and section_title not in already_exist_title:
            # print(elem.text)
            para_dict[section_title] = elem.text
            already_exist_title.add(section_title)
        elif elem.tag == section_paratag and section_title in already_exist_title:
#             print(section_title)
            existing_text = para_dict[section_title]
#             print(existing_text)
            if existing_text:
                if elem.text:
                    para_dict[section_title] = existing_text +  elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)

In [42]:
{k: v for k, v in actual_body_list[0].items() if k}

{'Introduction': 'Incomplete B cell depletion following treatment with the anti-CD20 mAb rituximab (RTX) is associated with poor clinical response in both RA [B cell subpopulations may be defined as naïve (IgDAnti-CD20 mAbs evoke distinct cytotoxic mechanisms: complement-dependent cellular cytotoxicity (CDC), Fc gamma receptor (FcγR)-mediated depletion through cellular effectors including antibody-dependent cellular cytotoxicity (ADCC) and antibody-dependent cellular phagocytosis, and direct cell death (DCD). Good clinical response to rituximab in both RA [Anti-CD20 mAbs can be categorized as types I and II. Type I anti-CD20 mAbs, such as RTX, redistribute CD20 into lipid rafts, a property that facilitates clustering and complement activation, but also mAb internalization, which is partly driven by cis-mediated engagement of FcγRIIb [Other type I mAbs, ofatumumab and ocrelizumab, have been used in clinical trial settings in RA and/or SLE, respectively [Our previous work showed that int

In [623]:
from xml.etree.ElementTree import XMLParser
list_to=[]
class MaxDepth:                     # The target object of the parser
    maxDepth = 0
    depth = 0
    def start(self, tag, attrib):   # Called for each opening tag.
        self.depth += 1
        if self.depth > self.maxDepth:
            self.maxDepth = self.depth
    def end(self, tag):             # Called for each closing tag.
        self.depth -= 1
    def data(self, data):
        list_to.append(data)            # We do not need to do anything with data.
    def close(self):    # Called when all data has been parsed.
        return self.maxDepth
target = MaxDepth()
parser = XMLParser(target=target)
parser.feed('<abstract><title>Abstract</title><p>\n<bold>Objective.</bold> A proportion of RA and SLE patients treated with standard doses of rituximab (RTX) display inefficient B cell deletion and poor clinical responses that can be augmented by delivering higher doses, indicating that standard-dose RTX is a sub-optimal therapy in these patients. This study aimed to investigate whether better responses could be achieved with mechanistically different anti-CD20 mAbs.</p><p>\n<bold>Methods.</bold> We compared RTX with obinutuzumab (OBZ), a new-generation, glycoengineered type II anti-CD20 mAb, in a series of <italic>in vitro</italic> assays measuring B cell cytotoxicity in RA and SLE patient samples.</p><p>\n<bold>Results.</bold> We found that OBZ was at least 2-fold more efficient than RTX at inducing B-cell cytotoxicity in <italic>in vitro</italic> whole blood assays. Dissecting this difference, we found that RTX elicited more potent complement-dependent cellular cytotoxicity than OBZ. In contrast, OBZ was more effective at evoking Fc gamma receptor-mediated effector mechanisms, including activation of NK cells and neutrophils, probably due to stronger interaction with Fc gamma receptors and the ability of OBZ to remain at the cell surface following CD20 engagement, whereas RTX became internalized. OBZ was also more efficient at inducing direct cell death. This was true for all CD19<sup>+</sup> B cells as a whole and in na&#239;ve (IgD<sup>+</sup>CD27<sup>&#8722;</sup>) and switched (IgD<sup>&#8722;</sup>CD27<sup>+</sup>) memory B cells specifically, a higher frequency of which is associated with poor clinical response after RTX.</p><p>\n<bold>Conclusion.</bold> Taken together, these data provide a mechanistic basis for resistance to rituximab-induced B-cell depletion, and for considering obinutuzumab as an alternative B-cell depleting agent in RA and SLE.</p></abstract>')


In [621]:
''.join(list_to).replace("\n","")

'AbstractObjective. A proportion of RA and SLE patients treated with standard doses of rituximab (RTX) display inefficient B cell deletion and poor clinical responses that can be augmented by delivering higher doses, indicating that standard-dose RTX is a sub-optimal therapy in these patients. This study aimed to investigate whether better responses could be achieved with mechanistically different anti-CD20 mAbs.Methods. We compared RTX with obinutuzumab (OBZ), a new-generation, glycoengineered type II anti-CD20 mAb, in a series of in vitro assays measuring B cell cytotoxicity in RA and SLE patient samples.Results. We found that OBZ was at least 2-fold more efficient than RTX at inducing B-cell cytotoxicity in in vitro whole blood assays. Dissecting this difference, we found that RTX elicited more potent complement-dependent cellular cytotoxicity than OBZ. In contrast, OBZ was more effective at evoking Fc gamma receptor-mediated effector mechanisms, including activation of NK cells and

## Abstract Extraction (NLM)

In [352]:
def get_from_abstract_NLM(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'NLM' in shorttext:
            #print(SourceName)
            
            tree = etree.parse(SourceName)
            roots = tree.getroot()

                
            for tag in roots.iter("abstract"):
                #print(tag)
                tagtext = ElementTree.tostring(tag)
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/abstract|abstract|p|/p|title|/title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                
                #print((tagtext))
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext)
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "title":
                        abstract_title = elem.text
                    if elem.tag == "p":
                        #print(elem.text)
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [353]:
get_from_abstract_NLM('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/02be68ce-7530-4882-9a02-d793e6306b43.xml')

{'Abstract': 'Objective. A proportion of RA and SLE patients treated with standard doses of rituximab (RTX) display inefficient B cell deletion and poor clinical responses that can be augmented by delivering higher doses, indicating that standard-dose RTX is a sub-optimal therapy in these patients. This study aimed to investigate whether better responses could be achieved with mechanistically different anti-CD20 mAbs. Methods. We compared RTX with obinutuzumab (OBZ), a new-generation, glycoengineered type II anti-CD20 mAb, in a series of in vitro assays measuring B cell cytotoxicity in RA and SLE patient samples. Results. We found that OBZ was at least 2-fold more efficient than RTX at inducing B-cell cytotoxicity in in vitro whole blood assays. Dissecting this difference, we found that RTX elicited more potent complement-dependent cellular cytotoxicity than OBZ. In contrast, OBZ was more effective at evoking Fc gamma receptor-mediated effector mechanisms, including activation of NK ce

## Other Sections Extraction (NLM)

In [354]:
def get_sections_from_body_NLM(SourceName):
    
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        list_to=[]
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'NLM' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            '''
            for elem in tree.xpath('//italic'):
                parent = elem.getparent()
                parent.remove(elem)
            for elem in tree.xpath('//sub'):
                parent = elem.getparent()
                parent.remove(elem)
            for elem in tree.xpath('//xref'):
                parent = elem.getparent()
                parent.remove(elem)
                #print(etree.tostring(doc))
            '''
            for tag in roots.iter("body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/body|body|p|/p|title|/title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "title":
                        section_title = elem.text
                        
                    if elem.tag == "p":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
        section_dict={k: v for k, v in section_dict.items() if k}                 
        return section_dict

In [355]:
get_sections_from_body_NLM('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/02be68ce-7530-4882-9a02-d793e6306b43.xml')

{'Introduction': 'Incomplete B cell depletion following treatment with the anti-CD20 mAb rituximab (RTX) is associated with poor clinical response in both RA [1] and SLE [2], whereas enhanced B cell depletion achieved using additional doses of RTX in RA [3] and prolonged duration of depletion are associated with a better clinical response in SLE [4]. Therefore, these data indicate that achieving complete, durable B cell depletion will improve clinical response in both RA and SLE. B cell subpopulations may be defined as naïve (IgD+CD27−), unswitched memory cells (IgD+CD27+), switched memory cells (IgD−CD27+) and double negative (DN) cells (IgD−CD27−). Poor clinical response to RTX in both RA and SLE is associated with a higher number and/or frequency of CD27+ memory cells [1, 2, 5] and also with DN B cells in RA [5], suggesting that resistance to depletion of different B cell subpopulations is clinically relevant [6]. Further, a greater frequency of IgD−CD27+ switched memory cells and D

### Code to Validate NCT from Abstract (NLM)

In [443]:
#To Get NCT from Abstract

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
check_list=[]
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                dd=NCT_filter(get_from_abstract_NLM(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #count=count+1

                    #To Verify publication NCT
                    for i in range(0,len(list(dd.keys()))):             
                        NCT_refresh=[] 
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                        NCT_refresh=' '.join(NCT_refresh).split()
                        #print(NCT_refresh)
                        if len(NCT_refresh)>0:
                            for search_word in NCT_refresh:
                                if len(search_word)==11:                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        xml_list.append(xml_name)
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        val_list.append('Abstract')
                                        check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                            
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    val_list.append('NA')
                    check_list.append('NA')
        df1_NLM =pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
            

In [444]:
#URL Extraction and Output
df1_NLM['URL_found']=df1_NLM['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df1_NLM.head()

XML Section Name  \
0  01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml           NA   
1  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml           NA   
2  02be68ce-7530-4882-9a02-d793e6306b43.xml           NA   
3  0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml           NA   
4  05294995-39f2-48b3-bbae-0a23ffad6c7f.xml           NA   

  NCT Number in Section Sentence Containing NCT Number Supporting word  \
0                    NA                             NA              NA   
1                    NA                             NA              NA   
2                    NA                             NA              NA   
3                    NA                             NA              NA   
4                    NA                             NA              NA   

  URL_found  
0        []  
1        []  
2        []  
3        []  
4        []

### Code to Validate NCT from Other Sections (NLM)

In [445]:
#To Get NCT from Body
import pandas as pd
import numpy as np

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
section_list=[]
check_list=[]

NCT_final_list={}
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(name)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                dd=NCT_filter(get_sections_from_body_NLM(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #To Verify publication NCT
                    NCT_total_list=[]
                    for i in range(0,len(list(dd.keys()))): 
                        #print(len(list(dd.keys())))
                        NCT_refresh=[] 
                        
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                        NCT_refresh=' '.join(NCT_refresh).split() 
                        #NCT_total_list.append(list(set(NCT_refresh)))
                        #print(NCT_refresh)
                        
                        if len(NCT_refresh)>0:
                            
                            for search_word in NCT_refresh:
                                if len(search_word)==11:
                                    #print(search_word)
                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        #print(proximity_text)
                                        #print(xml_name)
                                        xml_list.append(xml_name)
                                        section_list.append(list(dd.keys())[i])
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        
                                        check_words=['register','started','ongoing','underway'
                                                     ,'reported','ClinicalTrials.gov'
                                             ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                        ##val_list.append('Yes')
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                                    #NCT_final_list.append('NA')
                    NCT_dict={xml_name:NCT_total_list}
                    NCT_final_list.update(NCT_dict)
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    section_list.append('NA')
                    check_list.append('NA')
                    NCT_dict={xml_name:'NA'}
                    NCT_final_list.update(NCT_dict)
                    
            df2_NLM=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
#df2=df2.replace('NA',np.nan).dropna(subset=['Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number']).reset_index().drop(columns=['index'])      

In [446]:
#URL Extraction and Output
df2_NLM['URL_found']=df2_NLM['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df2_NLM.head()

XML                    Section Name  \
0  01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml                              NA   
1  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml  Therapies acting on complement   
2  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml  Therapies acting on complement   
3  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml  Therapies acting on complement   
4  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml  Therapies acting on complement   

  NCT Number in Section                     Sentence Containing NCT Number  \
0                    NA                                                 NA   
1           NCT03347396  recurred when drug levels were cleared and ree...   
2           NCT03347422  recurred when drug levels were cleared and ree...   
3           NCT03226678  pathways inhibitor), has shown a good efficacy...   
4           NCT03010046  ANX005, has been shown to block classical comp...   

  Supporting word URL_found  
0              NA        []  
1              NA        []  
2              NA        []  
3       [ongoing]        []  
4              NA        []

## Final Dataframe (NLM)

In [448]:
#To create JSON using data_to_json function
XML_files_unique = df1_NLM['XML'].unique()
final_df_NLM = data_to_json(df1_NLM,df2_NLM,'NCT_Json',XML_files_unique,'NLM')
final_df_NLM.head()

XML Publisher  \
0  01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml       NLM   
1  01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml       NLM   
2  02be68ce-7530-4882-9a02-d793e6306b43.xml       NLM   
3  0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml       NLM   
4  05294995-39f2-48b3-bbae-0a23ffad6c7f.xml       NLM   

                                            NCT_Json  \
0                                                      
1  {"Section Name":"Therapies acting on complemen...   
2  {"Section Name":"Discussion","NCT Number in Se...   
3  {"Section Name":"MEDI-551","NCT Number in Sect...   
4  {"Section Name":"Indirect Comparisons","NCT Nu...   

                                       NCT Full List  
0                                                     
1  [NCT03347396, NCT03347422, NCT03226678, NCT030...  
2                         [NCT02550652, NCT02586051]  
3  [NCT01585766, NCT00882999, NCT01051817, NCT018...  
4                                      [NCT02980042]

In [449]:
pd.DataFrame([json.loads(final_df_NLM['NCT_Json'].iloc[1].split(';;;;')[6])])

NCT Number in Section                                       Section Name  \
0           NCT03075878  Therapies acting on monocyte/macrophages and o...   

                      Sentence Containing NCT Number Supporting word URL_found  
0  in a recent first-in-human study [65]. A phase...              NA        []

### Extract Title and Journal Title with XML name (NLM)

In [37]:
#RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
#import xml.etree.ElementTree as etree
#import pandas as pd

def NLM_Titles_XML(RootDir1):
    xml_title_list=[]
    actual_title_list=[]
    actual_journal_list=[]

    for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    #print(SourceFolder)
                    tree = etree.parse(SourceFolder)
                    roots = tree.getroot()


                    for tag in roots.iter('front'):
                        tagtext = ElementTree.tostring(tag) 
                        #print(tagtext)
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/article-title|article-title|front|/front|journal-title|/journal-title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        #print(tagtext)
                        root_1 = ElementTree.fromstring(tagtext)               
                        title_sen=""
                        journal_sen=""
                        for elem in root_1.iter():
                            #print(elem)
                            if elem.tag == "article-title":                    
                                title_sen = title_sen+elem.text
                            if elem.tag =="journal-title":
                                journal_sen= journal_sen+elem.text

                        actual_journal_list.append(journal_sen)
                        actual_title_list.append(title_sen)
                        xml_title_list.append(xml_name)

    title_df_NLM = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
                   columns =['XML','Title','Journal Title'])
    return title_df_NLM

In [38]:
NLM_Titles_XML('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/')

XML  \
0    01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml   
1    01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml   
2    02be68ce-7530-4882-9a02-d793e6306b43.xml   
3    0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml   
4    05294995-39f2-48b3-bbae-0a23ffad6c7f.xml   
5    068bfff5-8882-4c9b-85ba-6d5dce67aa53.xml   
6    090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml   
7    0aadf051-3352-4a49-8744-631de0697f0a.xml   
8    0bc312b9-a280-4d2c-8ebe-92f48585707f.xml   
9    0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml   
10   0f2f6b6c-7f6d-4b64-92ca-a16999a09c71.xml   
11   15144c1e-5c1d-477e-b0db-437eebe23d97.xml   
12   15914e48-3869-47ec-8f82-f3b13638d8e5.xml   
13   15a5c64c-891b-415c-b706-76d1a45cbdf6.xml   
14   19636024-41d2-4607-86ca-c56892608622.xml   
15   1ba62e19-e0c5-4b43-8f31-01ec63e66ffb.xml   
16   1bd10a42-8aa8-49f4-bd4b-5e59ac175c0c.xml   
17   1c2d5b27-8bf9-476a-83b2-12f7e539bada.xml   
18   1f5ef9dd-d064-4888-babc-f09c721620ea.xml   
19   22adc16b-8685-42ff-aa91-d46f10eb55be.xml   
20   2468cc7b-4c08-46a3-97b4-104a9810cc36.xml   
21   251647c1-05e3-443b-8fbd-e3a01dac1d3c.xml   
22   28ee2704-fb96-4e01-8b1d-3b262b082623.xml   
23   311b274e-c6a7-4c21-bfdb-52664722abff.xml   
24   31bef979-d07b-4332-9438-53dc9a7e7df9.xml   
25   3280ec95-ba06-4699-852e-5eea9a09f9f8.xml   
26   3392759a-b97d-41ba-b077-69207e85bca1.xml   
27   3bca6af4-e13e-4df9-90ad-a5878eadd58e.xml   
28   3bcd9e40-ab01-498a-a86c-17b9971db667.xml   
29   3bcef5b4-c97d-4fb2-921b-44955d3c8156.xml   
..                                        ...   
99   bdae611a-da74-41e0-8e0b-8de922ef0e6c.xml   
100  bdc23e24-543f-4bdd-83be-ed7aa5928745.xml   
101  c3ada49d-0b6c-4664-9b67-25df5c41e290.xml   
102  c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml   
103  c723c457-acd1-4a43-be22-4a796692b495.xml   
104  c798564e-9105-45e4-b29a-73eb09b2b2fd.xml   
105  c7d988be-6699-4a96-8338-497b9984e784.xml   
106  ca148833-6462-410e-9898-bc995b01ad4a.xml   
107  cadcf7c3-3c90-4512-8ecc-e3d7f611cc55.xml   
108  cb8fc27c-c46e-4977-869e-6f344b26d223.xml   
109  cd23a882-bccd-4faa-aadf-73739e1bc736.xml   
110  d0fc120b-34c5-4c77-80df-1899801e59dd.xml   
111  d11309dc-a2fa-4ab6-818a-1e0590ecbc27.xml   
112  dc843218-8a0e-4946-b8b8-0ca4349db24f.xml   
113  ded024a8-3610-4f73-a29b-adec96685b33.xml   
114  dee575b2-2841-4a8f-8055-dfcb5418af48.xml   
115  e0195958-a9dd-47ac-ade8-46e2c93f48e8.xml   
116  e0bc7241-014a-40e3-a7fb-f43ab81b4502.xml   
117  e509b3bf-4e6b-48ee-8cfe-a92deb9771b9.xml   
118  e5b5b6a1-49e3-400f-bf93-4779aa4ad492.xml   
119  e60923f7-cbcc-4412-b7a0-9660e5f1b74b.xml   
120  ec819b82-c4fa-4eb4-a73c-d38376aa456b.xml   
121  ecdc1a17-0574-49be-a092-d7abca902752.xml   
122  f18576d9-ac93-45cb-adff-98efb35dc8b0.xml   
123  f19438e6-8ec2-48eb-a3cd-66d3b5b3dcdc.xml   
124  f2037c20-a62f-420c-a8b7-19837ddfac7c.xml   
125  f8e72c62-988e-4efc-8d07-c8a214c323a4.xml   
126  fb120bc9-b590-4877-aec0-daf8cebf04f3.xml   
127  ff4db333-e929-492d-b335-4e2092417000.xml   
128  ff825821-c3f4-45c4-87ed-82fef03c0ffb.xml   

                                                 Title  \
0    Early and unrestricted access to high-efficacy...   
1    Current and emerging treatment options for aut...   
2    Obinutuzumab induces superior B-cell cytotoxic...   
3    Recent Advances in Monoclonal Antibody Therapi...   
4    Rituximab in Multiple Sclerosis: Are We Ready ...   
5    Invasive Fungal Diseases in Children with Hema...   
6    Monoclonal Antibody Therapies for Hematologica...   
7    Development of humoral and cellular immunologi...   
8    Investigational drugs for treating psoriatic a...   
9    Alemtuzumab for the treatment of relapsing-rem...   
10   Current and emerging monoclonal antibody treat...   
11   Emerging biological therapies to treat acute l...   
12   PCI-32765: a novel Bruton's tyrosine kinase in...   
13   Differential Effects of MS Therapeutics on B C...   
14   Skin-Associated B Cells in the Pathogenesis of...   
15   What's New in the Treatment of Systemic Lupus ...   
16   Novel Molecu

In [673]:
title_df_NLM

XML  \
0    01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml   
1    01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml   
2    02be68ce-7530-4882-9a02-d793e6306b43.xml   
3    0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml   
4    05294995-39f2-48b3-bbae-0a23ffad6c7f.xml   
5    068bfff5-8882-4c9b-85ba-6d5dce67aa53.xml   
6    090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml   
7    0aadf051-3352-4a49-8744-631de0697f0a.xml   
8    0bc312b9-a280-4d2c-8ebe-92f48585707f.xml   
9    0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml   
10   0f2f6b6c-7f6d-4b64-92ca-a16999a09c71.xml   
11   15144c1e-5c1d-477e-b0db-437eebe23d97.xml   
12   15914e48-3869-47ec-8f82-f3b13638d8e5.xml   
13   15a5c64c-891b-415c-b706-76d1a45cbdf6.xml   
14   19636024-41d2-4607-86ca-c56892608622.xml   
15   1ba62e19-e0c5-4b43-8f31-01ec63e66ffb.xml   
16   1bd10a42-8aa8-49f4-bd4b-5e59ac175c0c.xml   
17   1c2d5b27-8bf9-476a-83b2-12f7e539bada.xml   
18   1f5ef9dd-d064-4888-babc-f09c721620ea.xml   
19   22adc16b-8685-42ff-aa91-d46f10eb55be.xml   
20   2468cc7b-4c08-46a3-97b4-104a9810cc36.xml   
21   251647c1-05e3-443b-8fbd-e3a01dac1d3c.xml   
22   28ee2704-fb96-4e01-8b1d-3b262b082623.xml   
23   311b274e-c6a7-4c21-bfdb-52664722abff.xml   
24   31bef979-d07b-4332-9438-53dc9a7e7df9.xml   
25   3280ec95-ba06-4699-852e-5eea9a09f9f8.xml   
26   3392759a-b97d-41ba-b077-69207e85bca1.xml   
27   3bca6af4-e13e-4df9-90ad-a5878eadd58e.xml   
28   3bcd9e40-ab01-498a-a86c-17b9971db667.xml   
29   3bcef5b4-c97d-4fb2-921b-44955d3c8156.xml   
..                                        ...   
99   bdae611a-da74-41e0-8e0b-8de922ef0e6c.xml   
100  bdc23e24-543f-4bdd-83be-ed7aa5928745.xml   
101  c3ada49d-0b6c-4664-9b67-25df5c41e290.xml   
102  c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml   
103  c723c457-acd1-4a43-be22-4a796692b495.xml   
104  c798564e-9105-45e4-b29a-73eb09b2b2fd.xml   
105  c7d988be-6699-4a96-8338-497b9984e784.xml   
106  ca148833-6462-410e-9898-bc995b01ad4a.xml   
107  cadcf7c3-3c90-4512-8ecc-e3d7f611cc55.xml   
108  cb8fc27c-c46e-4977-869e-6f344b26d223.xml   
109  cd23a882-bccd-4faa-aadf-73739e1bc736.xml   
110  d0fc120b-34c5-4c77-80df-1899801e59dd.xml   
111  d11309dc-a2fa-4ab6-818a-1e0590ecbc27.xml   
112  dc843218-8a0e-4946-b8b8-0ca4349db24f.xml   
113  ded024a8-3610-4f73-a29b-adec96685b33.xml   
114  dee575b2-2841-4a8f-8055-dfcb5418af48.xml   
115  e0195958-a9dd-47ac-ade8-46e2c93f48e8.xml   
116  e0bc7241-014a-40e3-a7fb-f43ab81b4502.xml   
117  e509b3bf-4e6b-48ee-8cfe-a92deb9771b9.xml   
118  e5b5b6a1-49e3-400f-bf93-4779aa4ad492.xml   
119  e60923f7-cbcc-4412-b7a0-9660e5f1b74b.xml   
120  ec819b82-c4fa-4eb4-a73c-d38376aa456b.xml   
121  ecdc1a17-0574-49be-a092-d7abca902752.xml   
122  f18576d9-ac93-45cb-adff-98efb35dc8b0.xml   
123  f19438e6-8ec2-48eb-a3cd-66d3b5b3dcdc.xml   
124  f2037c20-a62f-420c-a8b7-19837ddfac7c.xml   
125  f8e72c62-988e-4efc-8d07-c8a214c323a4.xml   
126  fb120bc9-b590-4877-aec0-daf8cebf04f3.xml   
127  ff4db333-e929-492d-b335-4e2092417000.xml   
128  ff825821-c3f4-45c4-87ed-82fef03c0ffb.xml   

                                                 Title  \
0    Early and unrestricted access to high-efficacy...   
1    Current and emerging treatment options for aut...   
2    Obinutuzumab induces superior B-cell cytotoxic...   
3    Recent Advances in Monoclonal Antibody Therapi...   
4    Rituximab in Multiple Sclerosis: Are We Ready ...   
5    Invasive Fungal Diseases in Children with Hema...   
6    Monoclonal Antibody Therapies for Hematologica...   
7    Development of humoral and cellular immunologi...   
8    Investigational drugs for treating psoriatic a...   
9    Alemtuzumab for the treatment of relapsing-rem...   
10   Current and emerging monoclonal antibody treat...   
11   Emerging biological therapies to treat acute l...   
12   PCI-32765: a novel Bruton's tyrosine kinase in...   
13   Differential Effects of MS Therapeutics on B C...   
14   Skin-Associated B Cells in the Pathogenesis of...   
15   What's New in the Treatment of Systemic Lupus ...   
16   Novel Molecu

In [714]:
df_to_merge_NLM = pd.merge(title_df_NLM,final_df_NLM)
df_to_merge_NLM

XML  \
0    01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml   
1    01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml   
2    02be68ce-7530-4882-9a02-d793e6306b43.xml   
3    0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml   
4    05294995-39f2-48b3-bbae-0a23ffad6c7f.xml   
5    068bfff5-8882-4c9b-85ba-6d5dce67aa53.xml   
6    090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml   
7    0aadf051-3352-4a49-8744-631de0697f0a.xml   
8    0bc312b9-a280-4d2c-8ebe-92f48585707f.xml   
9    0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml   
10   0f2f6b6c-7f6d-4b64-92ca-a16999a09c71.xml   
11   15144c1e-5c1d-477e-b0db-437eebe23d97.xml   
12   15914e48-3869-47ec-8f82-f3b13638d8e5.xml   
13   15a5c64c-891b-415c-b706-76d1a45cbdf6.xml   
14   19636024-41d2-4607-86ca-c56892608622.xml   
15   1ba62e19-e0c5-4b43-8f31-01ec63e66ffb.xml   
16   1bd10a42-8aa8-49f4-bd4b-5e59ac175c0c.xml   
17   1c2d5b27-8bf9-476a-83b2-12f7e539bada.xml   
18   1f5ef9dd-d064-4888-babc-f09c721620ea.xml   
19   22adc16b-8685-42ff-aa91-d46f10eb55be.xml   
20   2468cc7b-4c08-46a3-97b4-104a9810cc36.xml   
21   251647c1-05e3-443b-8fbd-e3a01dac1d3c.xml   
22   28ee2704-fb96-4e01-8b1d-3b262b082623.xml   
23   311b274e-c6a7-4c21-bfdb-52664722abff.xml   
24   31bef979-d07b-4332-9438-53dc9a7e7df9.xml   
25   3280ec95-ba06-4699-852e-5eea9a09f9f8.xml   
26   3392759a-b97d-41ba-b077-69207e85bca1.xml   
27   3bca6af4-e13e-4df9-90ad-a5878eadd58e.xml   
28   3bcd9e40-ab01-498a-a86c-17b9971db667.xml   
29   3bcef5b4-c97d-4fb2-921b-44955d3c8156.xml   
..                                        ...   
99   bdae611a-da74-41e0-8e0b-8de922ef0e6c.xml   
100  bdc23e24-543f-4bdd-83be-ed7aa5928745.xml   
101  c3ada49d-0b6c-4664-9b67-25df5c41e290.xml   
102  c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml   
103  c723c457-acd1-4a43-be22-4a796692b495.xml   
104  c798564e-9105-45e4-b29a-73eb09b2b2fd.xml   
105  c7d988be-6699-4a96-8338-497b9984e784.xml   
106  ca148833-6462-410e-9898-bc995b01ad4a.xml   
107  cadcf7c3-3c90-4512-8ecc-e3d7f611cc55.xml   
108  cb8fc27c-c46e-4977-869e-6f344b26d223.xml   
109  cd23a882-bccd-4faa-aadf-73739e1bc736.xml   
110  d0fc120b-34c5-4c77-80df-1899801e59dd.xml   
111  d11309dc-a2fa-4ab6-818a-1e0590ecbc27.xml   
112  dc843218-8a0e-4946-b8b8-0ca4349db24f.xml   
113  ded024a8-3610-4f73-a29b-adec96685b33.xml   
114  dee575b2-2841-4a8f-8055-dfcb5418af48.xml   
115  e0195958-a9dd-47ac-ade8-46e2c93f48e8.xml   
116  e0bc7241-014a-40e3-a7fb-f43ab81b4502.xml   
117  e509b3bf-4e6b-48ee-8cfe-a92deb9771b9.xml   
118  e5b5b6a1-49e3-400f-bf93-4779aa4ad492.xml   
119  e60923f7-cbcc-4412-b7a0-9660e5f1b74b.xml   
120  ec819b82-c4fa-4eb4-a73c-d38376aa456b.xml   
121  ecdc1a17-0574-49be-a092-d7abca902752.xml   
122  f18576d9-ac93-45cb-adff-98efb35dc8b0.xml   
123  f19438e6-8ec2-48eb-a3cd-66d3b5b3dcdc.xml   
124  f2037c20-a62f-420c-a8b7-19837ddfac7c.xml   
125  f8e72c62-988e-4efc-8d07-c8a214c323a4.xml   
126  fb120bc9-b590-4877-aec0-daf8cebf04f3.xml   
127  ff4db333-e929-492d-b335-4e2092417000.xml   
128  ff825821-c3f4-45c4-87ed-82fef03c0ffb.xml   

                                                 Title  \
0    Early and unrestricted access to high-efficacy...   
1    Current and emerging treatment options for aut...   
2    Obinutuzumab induces superior B-cell cytotoxic...   
3    Recent Advances in Monoclonal Antibody Therapi...   
4    Rituximab in Multiple Sclerosis: Are We Ready ...   
5    Invasive Fungal Diseases in Children with Hema...   
6    Monoclonal Antibody Therapies for Hematologica...   
7    Development of humoral and cellular immunologi...   
8    Investigational drugs for treating psoriatic a...   
9    Alemtuzumab for the treatment of relapsing-rem...   
10   Current and emerging monoclonal antibody treat...   
11   Emerging biological therapies to treat acute l...   
12   PCI-32765: a novel Bruton's tyrosine kinase in...   
13   Differential Effects of MS Therapeutics on B C...   
14   Skin-Associated B Cells in the Pathogenesis of...   
15   What's New in the Treatment of Systemic Lupus ...   
16   Novel Molecu

## NLM Other columns extraction

In [ ]:
#extraction for NLM

import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'


#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "title"
section_paratag = "p"
bodycount = 0 
full_body_text = []
body_list = []
doi_list=[]
doicount=0

nlm_body_list = []
nlm_bodycount = 0
full_body_text_nlm = []
nlm_abstract_list = []
nlm_abstract_count = 0
full_abstract_text = []

tagname1 = 'body' 
tagname2 = 'abstract' 
tagname3 = 'ext-link'
tagname4 = 'ext-link'
tagname5 = 'article-title'
file_title = []
# elements = set()
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)       
        with open(SourceFolder) as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                tree = etree.parse(SourceFolder)           
                roots = tree.getroot()           
                # get body

                for tag in roots.iter(tagname1):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_body_text_nlm.append(' '.join(pageText))
                    nlm_body_list.append(ElementTree.tostring(tag))
                    nlm_bodycount = bodycount + 1

                # get abstract

                for tag in roots.iter(tagname2):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_abstract_text.append(' '.join(pageText))
                    nlm_abstract_list.append(ElementTree.tostring(tag))
                    nlm_abstract_count = nlm_abstract_count + 1

                #get doi   

                for tag in roots.iter(tagname3):
                    doi_list.append('None')
    #                 try:
    #                     doi_list.append(ElementTree.tostring(tag))
                    doicount = doicount + 1
    #                 except:
    #                     doi_list.append("None")
    #                     doicount = doicount + 1

                #get url

                for tag in roots.iter(tagname4):
                    # ElementTree.dump(tag)
    #                 try:
    #                     url_list.append(ElementTree.tostring(tag))
    #                     urlcount = urlcount + 1
    #                 # print(".....")
    #                 # print(tag.text)
    #                 # print("...")
    #                 except:
                    url_list.append("None")
                    urlcount = urlcount + 1

                #get title
                for tag in roots.iter(tagname5):
                    # ElementTree.dump(tag)
                    title_list.append(ElementTree.tostring(tag))

                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    titlecount = titlecount + 1

                title_list = []
                for elem in roots.iter():
    #                 elements.add(elem.tag)
                    if elem.tag == tagname5:
                        title_list.append(elem.text)

                file_title.append(title_list[0])


                
# #actual title
# count = 0
# actual_title_list = []
# for body in title_list:
#     root = ElementTree.fromstring(body)
#     # ElementTree.dump(root)
#     for elem in root.iter():
#         # print(elem.text)
#         count = count + 1
#         actual_title_list.append(elem.text)

#actual doi
count = 0
actual_doi_list = []
for body in doi_list:
#     root = ElementTree.fromstring(body)
#     # ElementTree.dump(root)
#     for elem in root.iter():
#         # print(elem.text)
#         count = count + 1
        actual_doi_list.append('None')

        
#actual url
count = 0
actual_url_list = []
for body in url_list:
#     root = ElementTree.fromstring(body)
#     # ElementTree.dump(root)
#     for elem in root.iter():
#         # print(elem.text)
#         count = count + 1
        actual_url_list.append('None')
        


#actual body 
for body in nlm_body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    already_exist_title = set()
    count =0
    existing_text = ""
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
#             print("_____")
#             print(elem.text)
#             print("_____")
            section_title = elem.text            
        if elem.tag == section_paratag and section_title not in already_exist_title:
            # print(elem.text)
            para_dict[section_title] = elem.text
            already_exist_title.add(section_title)
        elif elem.tag == section_paratag and section_title in already_exist_title:
#             count = count + 1 
#             print(section_title)
            existing_text = para_dict[section_title]
#             print(existing_text)
            if existing_text:
                if elem.text:
                    para_dict[section_title] = existing_text +  elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1



df_nlm_p1 = pd.DataFrame(list(zip(file_title, actual_body_list, actual_doi_list, actual_url_list,full_body_text_nlm,abstract_list,abstract_fulltext)),
               columns =['Title', 'Body', 'DOI','URL','Full_body_text','Abstract','Abstract_full'])
# str(my_dict)
def getFullText(text):
    return str(text)

df_nlm_p1['full_text'] = df_nlm_p1.Body.apply(getFullText)

df_nlm_p2 = pd.concat([df_nlm_p1,df_nlm_p1['Body'].apply(pd.Series)], axis=1)

df_nlm_p3 = pd.concat([df_nlm_p2,df_nlm_p2['Abstract'].apply(pd.Series)], axis=1)

df_nlm_Ofatumumab = df_nlm_p3[['Title','Full_body_text','Abstract_full','DOI','URL']]
df_nlm_Ofatumumab.shape

In [ ]:
df_nlm_Ofatumumab.head()

## NLM Dataframe

In [ ]:
pd.merge(df_nlm_Ofatumumab,df_to_merge_NLM)

# Wiley

### Abstract Function (Wiley)

In [624]:
def get_from_abstract_Wiley(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.wiley.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.wiley.com/namespaces/wiley}abstract"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/ns0:abstract|ns0:abstract|ns0:p|/ns0:p|ns0:title|/ns0:title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                #print(tagtext)
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    #if elem.tag == "{http://www.wiley.com/namespaces/wiley}section-title":
                    abstract_title = 'Abstract'
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}p":
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [625]:
#Dictionary created by above 'get_from_abstract' function
abs_dict=get_from_abstract_Wiley('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/f6331576-9051-45b2-aa23-68b826197125.xml')
abs_dict

{'Abstract': 'For decades, B cells were ignored in multiple sclerosis (MS) pathogenesis, and the disease was always regarded as a T cell‐mediated disorder. Recent evidence shows that there is an antigen‐driven B‐cell response in the central nervous system of patients with MS, and memory B cells/plasma cells are detectable in MS lesions. The striking efficacy of B cell‐depleting therapies in reducing the inflammatory activity of the disease highlights that B cells may play more pathogenetic roles than expected. B cells express several unique characteristic markers on their surface, for example, CD19, CD20 molecules, that provide selective targets for monoclonal antibodies. In this respect, several B cell‐targeted therapies emerged, including anti‐CD20 antibodies (rituximab, ocrelizumab, and ofatumumab), anti‐CD19 antibody (inebilizumab), and agents targeting the BAFF/APRIL signaling pathway (atacicept, belimumab, and LY2127399). In this review, we discuss, in detail, the immunobiology o

### Other Sections Function (Wiley)

In [626]:
def get_sections_from_body_Wiley(SourceName):
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.wiley.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.wiley.com/namespaces/wiley}body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/ns0:body|ns0:body|ns0:p|/ns0:p|ns0:title|/ns0:title|/table|table).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                #print(tagtext)
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":
                        section_title = elem.text
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}p":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
                            
        return section_dict

In [627]:
#Dictionary created by above 'get_sections_from_body' function
section_dict=get_sections_from_body_Wiley('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/f6331576-9051-45b2-aa23-68b826197125.xml')
section_dict

{'INTRODUCTION': 'Multiple sclerosis (MS) is a common debilitating disease of the central nervous system (CNS) that is long believed to be of an autoimmune origin. Until recently, owing to extensive analogy to T cell‐mediated experimental autoimmune encephalitis (EAE) model, the pathogenic role of B cell and its progeny in MS remained largely ignored. However, several evidence gained recently highlights that B cells may be involved in MS more than expected. First, most of the CNS demyelinated lesions, especially lesions obtained from patients with established MS, harbor B cells and plasma cells expressing hypermutated immunoglobulins (Ig), showing a local CNS Ag‐driven activation of B cells. Second, the dramatic therapeutic effect of B cell‐depleting agents in patients with MS. Third, the presence of oligoclonal Ig bands of high sensitivity for MS diagnosis. In this regard, it has been suggested that B cells may exert their deleterious effects in MS through several ways: Secretion of a

### Code to Validate NCT from Abstract (Wiley)

In [573]:
#To Get NCT from Abstract

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
check_list=[]
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.wiley.com' in shorttext:
                dd=NCT_filter(get_from_abstract_Wiley(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #count=count+1

                    #To Verify publication NCT
                    for i in range(0,len(list(dd.keys()))):             
                        NCT_refresh=[] 
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                        NCT_refresh=' '.join(NCT_refresh).split()
                        #print(NCT_refresh)
                        if len(NCT_refresh)>0:
                            for search_word in NCT_refresh:
                                if len(search_word)==11:                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        xml_list.append(xml_name)
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        val_list.append('Abstract')
                                        check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                            
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    val_list.append('NA')
                    check_list.append('NA')
            df1_wiley=pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object).drop_duplicates()
            

In [574]:
#URL Extraction and Output
df1_wiley['URL_found']=df1_wiley['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df1_wiley.head()

XML Section Name  \
0  0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml           NA   
1  0dda512f-64cc-471d-8299-7f3156f05b4c.xml           NA   
2  1db7d591-da69-403a-80e4-8e8d4533c747.xml           NA   
3  327d7412-5e45-43cd-b725-282660bb2fe8.xml           NA   
4  3d888bae-1c8d-42cb-973a-508f6691ce89.xml           NA   

  NCT Number in Section Sentence Containing NCT Number Supporting word  \
0                    NA                             NA              NA   
1                    NA                             NA              NA   
2                    NA                             NA              NA   
3                    NA                             NA              NA   
4                    NA                             NA              NA   

  URL_found  
0        []  
1        []  
2        []  
3        []  
4        []

### Code to Validate NCT from Other Sections (Wiley)

In [580]:
#To Get NCT from Body
import pandas as pd
import numpy as np

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
count=0
xml_list=[]
proximity_list=[]
search_list=[]
val_list=[]
section_list=[]
check_list=[]

NCT_final_list={}
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(name)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.wiley.com' in shorttext:
                dd=NCT_filter(get_sections_from_body_Wiley(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                #print(dd)
                if len(dd)>0:
                    #To Verify publication NCT
                    NCT_total_list=[]
                    for i in range(0,len(list(dd.keys()))): 
                        #print(len(list(dd.keys())))
                        NCT_refresh=[] 
                        
                        NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                        NCT_refresh=' '.join(NCT_refresh).split() 
                        #NCT_total_list.append(list(set(NCT_refresh)))
                        #print(NCT_refresh)
                        
                        if len(NCT_refresh)>0:
                            
                            for search_word in NCT_refresh:
                                if len(search_word)==11:
                                    #print(search_word)
                                    
                                    for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                        proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                         +" "+search_word+" "+
                                                         ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                        #print(proximity_text)
                                        #print(xml_name)
                                        xml_list.append(xml_name)
                                        section_list.append(list(dd.keys())[i])
                                        proximity_list.append(proximity_text)
                                        search_list.append(search_word)
                                        
                                        check_words=['register','started','ongoing','underway'
                                                     ,'reported','ClinicalTrials.gov'
                                             ,'identifier','under evaluation']
                                        NCT_verified=[]
                                        for check_word in check_words:
                                            if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                NCT_verified.append(check_word)
                                        if len(list(set(NCT_verified)))>0:
                                            check_list.append(list(set(NCT_verified)))
                                        else:
                                            check_list.append('NA')
                                        ##val_list.append('Yes')
                                else:
                                    xml_list.append(xml_name)
                                    section_list.append('NA')
                                    proximity_list.append('NA')
                                    search_list.append('NA')
                                    check_list.append('NA')
                                    #NCT_final_list.append('NA')
                    NCT_dict={xml_name:NCT_total_list}
                    NCT_final_list.update(NCT_dict)
                else:
                    xml_list.append(xml_name)
                    proximity_list.append('NA')
                    search_list.append('NA')
                    section_list.append('NA')
                    check_list.append('NA')
                    NCT_dict={xml_name:'NA'}
                    NCT_final_list.update(NCT_dict)
                    
            df2_wiley=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
               columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
#df2=df2.replace('NA',np.nan).dropna(subset=['Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number']).reset_index().drop(columns=['index'])      

In [581]:
#URL Extraction and Output
df2_wiley['URL_found']=df2_wiley['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
df2_wiley.head()

XML  \
0  0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml   
1  0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
2  0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
3  0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
4  1db7d591-da69-403a-80e4-8e8d4533c747.xml   

                              Section Name NCT Number in Section  \
0                                       NA                    NA   
1  Pembrolizumab and checkpoint inhibition           NCT04781855   
2  Pembrolizumab and checkpoint inhibition           NCT04271956   
3     CURRENT CHALLENGES AND UNCERTAINTIES           NCT04608318   
4                                       NA                    NA   

                      Sentence Containing NCT Number Supporting word URL_found  
0                                                 NA              NA        []  
1  with RT. As the efficacy of monotherapy does n...              NA        []  
2  the efficacy of monotherapy does not appear su...              NA        []  
3  of the two paradigms creates the greatest bene...       [ongoing]        []  
4                                                 NA              NA        []

### Final Dataframe (Wiley)

In [583]:
#To create JSON using data_to_json function
XML_files_unique = df1_wiley['XML'].unique()
final_df_Wiley = data_to_json(df1_wiley,df2_wiley,'NCT_Json',XML_files_unique,'Wiley')
final_df_Wiley.head()

XML Publisher  \
0  0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml     Wiley   
1  0dda512f-64cc-471d-8299-7f3156f05b4c.xml     Wiley   
2  1db7d591-da69-403a-80e4-8e8d4533c747.xml     Wiley   
3  327d7412-5e45-43cd-b725-282660bb2fe8.xml     Wiley   
4  3d888bae-1c8d-42cb-973a-508f6691ce89.xml     Wiley   

                                            NCT_Json  \
0                                                      
1  {"Section Name":"Pembrolizumab and checkpoint ...   
2                                                      
3  {"Section Name":"Data collection","NCT Number ...   
4  {"Section Name":"Patient Recruitment, Randomiz...   

                             NCT Full List  
0                                           
1  [NCT04781855, NCT04271956, NCT04608318]  
2                                           
3                            [NCT04355611]  
4                            [NCT00050245]

### Extract Title and Journal Title with XML name (Wiley)

In [703]:
RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'
import xml.etree.ElementTree as etree
import pandas as pd


xml_title_list=[]
actual_title_list=[]
actual_journal_list=[]

for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            title_list=[]
            journal_list=[]
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.wiley.com' in shorttext:
                #print(SourceFolder)
                tree = etree.parse(SourceFolder)
                roots = tree.getroot()

                for tag in roots.iter('{http://www.wiley.com/namespaces/wiley}contentMeta'):
                    tagtext = ElementTree.tostring(tag) 
                    #print(tagtext)
                    tagtext= (str(tagtext, 'UTF-8'))
                    cleanr = re.compile(r'(<(?!/ns0:contentMeta|ns0:contentMeta|ns0:publicationMeta|/ns0:publicationMeta|ns0:header|/ns0:header|ns0:title|/ns0:title).*?>)|(\n)')
                    tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                    #print(tagtext)
                    root_1 = ElementTree.fromstring(tagtext)               

                    for elem in root_1.iter():
                        if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":                    
                            title_list.append(elem.text)


                for tag in roots.iter('{http://www.wiley.com/namespaces/wiley}publicationMeta'):
                    tagtext = ElementTree.tostring(tag) 
                    #print(tagtext)
                    tagtext= (str(tagtext, 'UTF-8'))
                    cleanr = re.compile(r'(<(?!/ns0:contentMeta|ns0:contentMeta|ns0:publicationMeta|/ns0:publicationMeta|ns0:header|/ns0:header|ns0:title|/ns0:title).*?>)|(\n)')
                    tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                    #print(tagtext)
                    root_1 = ElementTree.fromstring(tagtext)               

                    for elem in root_1.iter():
                        if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":                    
                            journal_list.append(elem.text)
                        

                actual_journal_list.append(journal_list[0])
                actual_title_list.append(title_list[0])
                xml_title_list.append(xml_name)

title_df_wiley = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
               columns =['XML','Title','Journal Title'])

In [705]:
title_df_wiley.head()

XML  \
0  0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml   
1  0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
2  1db7d591-da69-403a-80e4-8e8d4533c747.xml   
3  327d7412-5e45-43cd-b725-282660bb2fe8.xml   
4  3d888bae-1c8d-42cb-973a-508f6691ce89.xml   

                                               Title  \
0  Beyond Randomized Clinical Trials: Use of Exte...   
1  Chronic lymphocytic leukemia: 2022 update on d...   
2  Ofatumumab, the first human anti‐CD20 monoclon...   
3  Outcomes of coronavirus disease 2019 in patien...   
4  A double‐blind, placebo‐controlled study of ri...   

                                Journal Title  
0        Clinical Pharmacology & Therapeutics  
1              American Journal of Hematology  
2  Annals of the New York Academy of Sciences  
3               European Journal of Neurology  
4                         Annals of Neurology

In [709]:
df_to_merge_wiley = pd.merge(title_df_wiley,final_df_Wiley)
df_to_merge_wiley

XML  \
0   0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml   
1   0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
2   1db7d591-da69-403a-80e4-8e8d4533c747.xml   
3   327d7412-5e45-43cd-b725-282660bb2fe8.xml   
4   3d888bae-1c8d-42cb-973a-508f6691ce89.xml   
5   3ec7266b-2cf2-4abd-8ae0-b8c7de0c1b97.xml   
6   3f4459c8-75db-4947-8d67-56f43e312ade.xml   
7   40d6cd65-f0f7-4bf5-8e34-d4f0526e6785.xml   
8   43e72596-c148-4ae1-9f50-cb7c891f3fa9.xml   
9   46dce873-dc55-4035-9fbb-91659db416ea.xml   
10  48667507-2430-4a22-a2c8-e9a2b881f86c.xml   
11  515b8e6a-a467-4c57-b68b-e8d86562bb28.xml   
12  52fa1ade-c0f3-476a-a1b2-91242bc736ee.xml   
13  534cbdbf-ce82-472e-a89f-975d010b89ca.xml   
14  540651af-e108-4366-b987-1ac669524edc.xml   
15  54e191e9-f777-4f62-a711-542ee2262cc2.xml   
16  5d36c664-61ad-4386-96ec-ebfc167db2a3.xml   
17  634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18  6571c94f-5207-46ab-aa3a-81e06389ee06.xml   
19  6ed15061-93e2-48bd-9bfa-ded9b54d1691.xml   
20  7297f07c-497b-4425-b1dd-177eb3dda1c3.xml   
21  7c046239-703e-47de-90e4-98e4d3cb91c9.xml   
22  8324e21b-ca0c-4ad3-bd6f-7a09becb047e.xml   
23  89d80693-2b17-43b9-ac53-9562ceb25948.xml   
24  93587271-2c84-494b-9a41-682faabe47b2.xml   
25  a8c2531a-62c8-4888-9976-7da1fab5348f.xml   
26  af27a976-c99b-4fe1-bf5e-bbdeb1797611.xml   
27  b42fc71b-a626-4019-87d0-fa364bf88e7d.xml   
28  b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
29  c3acc47f-7596-43d8-97a6-b2827bb2ebce.xml   
30  c4369451-091d-47d4-90e9-0f3c108b89aa.xml   
31  c6321da9-b781-482b-87d8-6dd832c630a5.xml   
32  c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
33  cb74abed-5ed4-4029-9117-078efbce9780.xml   
34  ce310c5a-de31-495c-8f93-ce59a11f4bbf.xml   
35  cf43627e-0109-4bd1-bdae-9c311b8fc5de.xml   
36  d1f8f853-3a1b-493b-920e-609ca5630ee4.xml   
37  d70a6729-b105-4ca6-89da-e4ea1981ebee.xml   
38  e552c93c-8f64-4ca5-8e15-def481a5c29b.xml   
39  e57e9944-53ef-4be9-8a45-2ec3dfd9d1a7.xml   
40  ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
41  f6331576-9051-45b2-aa23-68b826197125.xml   
42  f87a9d96-e2f0-4413-9cab-5361f9d152ec.xml   
43  fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   

                                                Title  \
0   Beyond Randomized Clinical Trials: Use of Exte...   
1   Chronic lymphocytic leukemia: 2022 update on d...   
2   Ofatumumab, the first human anti‐CD20 monoclon...   
3   Outcomes of coronavirus disease 2019 in patien...   
4   A double‐blind, placebo‐controlled study of ri...   
5   Anti‐CD20 monoclonal antibodies and their use ...   
6   Beyond Randomized Clinical Trials: Use of Exte...   
7   Autoimmune cytopenias in chronic lymphocytic l...   
8   Ofatumumab‐associated acute respiratory manife...   
9       Challenges in the pursuit of immune tolerance   
10  The IUPHAR Guide to Immunopharmacology: connec...   
11  Myasthenia gravis and specific immunotherapy: ...   
12  Chronic lymphocytic leukemia: 2022 update on d...   
13  Treatment of systemic lupus erythematosus: new...   
14  Is There a Place for Chimeric Antigen Receptor...   
15     Novel Multiple Sclerosis Drugs in the Pipeline   
16  Blinded continuous monitoring in clinical tria...   
17  Current and emerging disease‐modulatory therap...   
18  Population pharmacokinetics of ofatumumab in p...   
19  Characteristics and follow‐up of postmarketing...   
20  Developing the next generation of monoclonal a...   
21  From bench to bedside: evolving therapeutic ta...   
22  Is There a Place for Chimeric Antigen Receptor...   
23  A changing treatment landscape for multiple sc...   
24  SARS‐CoV‐2 and Multiple Sclerosis: Not All Imm...   
25  Modulating inflammation and neuroprotection in...   
26  New era of therapy for endocrine autoimmune di...   
27                 DMDs in neuroimmunological disease   
28  Therapeutic Application of Monoclonal Antibodi...   
29  Ocrelizumab in relapsing and primary progressi...   
30                         MS treatment and infection   
31  Assessment of the effect of ofatumumab on card...   

### Wiley Other columns extraction

In [ ]:
#extraction of wiley

RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/'

#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "{http://www.wiley.com/namespaces/wiley}title"
section_paratag = "{http://www.wiley.com/namespaces/wiley}p"
bodycount = 0 
full_body_text = []
body_list = []


wiley_body_list = []
wiley_bodycount = 0
full_body_text_wiley = []
file_title = []
count = 0
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name) 
#         print(SourceFolder)
        try:
            with open(SourceFolder) as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:200]
                if 'www.wiley.com' in shorttext:
                    tree = etree.parse(SourceFolder)  
#                     print(SourceFolder)
    #                 count = count +1 
                    roots = tree.getroot()
                    tagname = '{http://www.wiley.com/namespaces/wiley}body' 
                    for tag in roots.iter(tagname):
                        tagtext = ElementTree.tostring(tag)   
                        soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                        pageText = soup.findAll(text=True)
                        full_body_text_wiley.append(' '.join(pageText))
                        wiley_body_list.append(ElementTree.tostring(tag))
                        wiley_bodycount = wiley_bodycount + 1


                     #get title

                    title_list = []
                    tagname5 = '{http://www.wiley.com/namespaces/wiley}articleTitle'
                    for elem in roots.iter():
        #                 elements.add(elem.tag)
                        if elem.tag == tagname5:
                            title_list.append(elem.text)

                    file_title.append(title_list[0])

        except:
            check = 2
            
        

# print(count)
for body in wiley_body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    already_exist_title = set()
    count =0
    existing_text = ""
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
#             print("_____")
#             print(elem.text)
#             print("_____")
            section_title = elem.text            
        if elem.tag == section_paratag and section_title not in already_exist_title:
            # print(elem.text)
            para_dict[section_title] = elem.text
            already_exist_title.add(section_title)
        elif elem.tag == section_paratag and section_title in already_exist_title:
#             count = count + 1 
#             print(section_title)
            existing_text = para_dict[section_title]
#             print(existing_text)
            if existing_text:
                if elem.text:
                    para_dict[section_title] = existing_text +  elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1

# #actual doi
# count = 0
# actual_doi_list = []
# for body in doi_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_doi_list.append('None')

        
# #actual url
# count = 0
# actual_url_list = []
# for body in url_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_url_list.append('None')
    
# #actual abstract
# count = 0
# actual_abstract_list = []
# for body in url_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_abstract_list.append('None')


df_wiley = pd.DataFrame(list(zip(file_title, actual_body_list, full_body_text_wiley)),
               columns =['Title','Body','Full_body_text'])
# str(my_dict)
def getFullText(text):
    return str(text)

df_wiley['full_text'] = df_wiley.Body.apply(getFullText)

df_wiley = pd.concat([df_wiley,df_wiley['Body'].apply(pd.Series)], axis=1)

df_wiley_Ofatumumab = df_wiley[['Title','Full_body_text']]
df_wiley_Ofatumumab.shape

In [ ]:
df_wiley_Ofatumumab.head()

##  Wiley Dataframe

In [ ]:
Wiley_data = pd.merge(df_elsiver_Ofatumumab,df_to_merge_wiley)
Wiley_data

# ------------------------------------------------------------

In [514]:
sddd=pd.concat([df2_elsevier,df1_elsevier]).sort_values('XML')
sddd=sddd.reset_index().drop(columns=['index'])
sddd

XML          Section Name  \
0    014a758a-0be7-4af8-a0f0-1291f198d303.xml                    NA   
1    014a758a-0be7-4af8-a0f0-1291f198d303.xml                    NA   
2    099b9fa2-e041-491c-b486-1a3d534e2eee.xml              Outcomes   
3    099b9fa2-e041-491c-b486-1a3d534e2eee.xml              Outcomes   
4    099b9fa2-e041-491c-b486-1a3d534e2eee.xml            Discussion   
5    099b9fa2-e041-491c-b486-1a3d534e2eee.xml              Abstract   
6    099b9fa2-e041-491c-b486-1a3d534e2eee.xml              Outcomes   
7    099b9fa2-e041-491c-b486-1a3d534e2eee.xml  Statistical analysis   
8    099b9fa2-e041-491c-b486-1a3d534e2eee.xml              Outcomes   
9    0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml                    NA   
10   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml                    NA   
11   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml                    NA   
12   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml                    NA   
13   176db363-c021-4d29-87c7-023169200fdf.xml                    NA   
14   176db363-c021-4d29-87c7-023169200fdf.xml                    NA   
15   1eda660e-600f-4f98-9758-137b9878e7b7.xml                    NA   
16   1eda660e-600f-4f98-9758-137b9878e7b7.xml                    NA   
17   28abfbe3-6332-4df1-ac04-88f33643b81e.xml                    NA   
18   28abfbe3-6332-4df1-ac04-88f33643b81e.xml                    NA   
19   30c3d283-6acd-409c-91c1-952fffdc8515.xml                    NA   
20   30c3d283-6acd-409c-91c1-952fffdc8515.xml                    NA   
21   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml                    NA   
22   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml                    NA   
23   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml                    NA   
24   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml                    NA   
25   3dd44642-f13a-4cca-83cd-ab262455c5a2.xml                    NA   
26   3dd44642-f13a-4cca-83cd-ab262455c5a2.xml                    NA   
27   3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml                    NA   
28   3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml                    NA   
29   4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml            Discussion   
..                                        ...                   ...   
123  ce1115df-1248-419f-a246-e1f6676556bd.xml                    NA   
124  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml                    NA   
125  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml                    NA   
126  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml                    NA   
127  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml                    NA   
128  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml                    NA   
129  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml                    NA   
130  da32c93e-b332-4e10-b07c-6be7035d575b.xml              Outcomes   
131  da32c93e-b332-4e10-b07c-6be7035d575b.xml              Abstract   
132  da32c93e-b332-4e10-b07c-6be7035d575b.xml              Outcomes   
133  da32c93e-b332-4e10-b07c-6be7035d575b.xml              Outcomes   
134  da32c93e-b332-4e10-b07c-6be7035d575b.xml  Statistical analysis   
135  da32c93e-b332-4e10-b07c-6be7035d575b.xml              Outcomes   
136  da32c93e-b332-4e10-b07c-6be7035d575b.xml            Discussion   
137  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml                    NA   
138  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml                    NA   
139  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml                    NA   
140  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml                    NA   
141  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml                    NA   
142  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml                    NA   
143  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml                    NA   
144  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml                    NA   
145  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml                    NA   
146  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml                    NA   
147  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml                    N

In [528]:
writer = pd.ExcelWriter('C:/Users/GOWRICH1/Downloads/HAWK AI/Elsevier_data1.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
pd.merge(Elsevier_data[['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Publisher','NCT Full List']],sdff,on='XML').to_excel(writer)
writer.close()

In [525]:
Elsevier_data.columns

Index(['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Publisher', 'NCT_Json', 'NCT Full List'],
      dtype='object')

In [520]:
sdff= pd.read_excel('C:/Users/GOWRICH1/Downloads/HAWK AI/Elsevier_data1.xlsx')
sdff=sdff.drop_duplicates().reset_index().drop(columns=['index'])
sdff

XML  \
0    014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
3    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
4    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
5    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
6    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
7    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
8    0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
9    0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
10   176db363-c021-4d29-87c7-023169200fdf.xml   
11   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
12   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
13   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
14   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
15   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
16   3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
17   3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
18   4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
19   4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
20   427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
21   427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
22   42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
23   44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
24   4c527b19-40c1-4b09-812c-7817324e4e66.xml   
25   5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
26   52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
27   5304e59e-0eaf-480e-a51a-04139abd9913.xml   
28   53734523-9df0-460f-972b-3cd7934e1169.xml   
29   5445f1b8-9ded-478a-a835-076bf5758808.xml   
..                                        ...   
71   bbc69df3-927a-4878-b029-a368b8834b35.xml   
72   bbc69df3-927a-4878-b029-a368b8834b35.xml   
73   be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
74   c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
75   c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
76   c1417907-53ee-4d31-a753-cd008516c324.xml   
77   c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
78   c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
79   c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
80   c512c537-32fb-46b6-adaa-4589799569bc.xml   
81   cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
82   ce1115df-1248-419f-a246-e1f6676556bd.xml   
83   d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
84   d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
85   da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
86   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
87   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
88   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
89   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
90   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
91   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
92   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
93   dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
94   e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
95   e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
96   ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
97   f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
98   f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
99   fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
100  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                         Section Name NCT Number in Section  \
0                                                 NaN                   NaN   
1                                            Outcomes           NCT04410991   
2                                            Outcomes           NCT04410978   
3                                          Discussion           NCT03996291   
4                                            Abstract           NCT03889639   
5                                            Outcomes           NCT04458051   
6                                Statistical analysis           NCT03889639   
7                                            Outcomes           NCT04411641   
8                                                 NaN                   NaN   
9                                                 NaN                   NaN   
10                                                NaN                   NaN   
11                                                NaN  

In [527]:
new_df=pd.merge(Elsevier_data[['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Publisher','NCT Full List']],sdff,on='XML')

XML  \
0    014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
3    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
4    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
5    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
6    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
7    099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
8    0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
9    0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
10   176db363-c021-4d29-87c7-023169200fdf.xml   
11   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
12   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
13   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
14   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
15   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
16   3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
17   3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
18   4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
19   4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
20   427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
21   427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
22   42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
23   44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
24   4c527b19-40c1-4b09-812c-7817324e4e66.xml   
25   5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
26   52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
27   5304e59e-0eaf-480e-a51a-04139abd9913.xml   
28   53734523-9df0-460f-972b-3cd7934e1169.xml   
29   5445f1b8-9ded-478a-a835-076bf5758808.xml   
..                                        ...   
71   bbc69df3-927a-4878-b029-a368b8834b35.xml   
72   bbc69df3-927a-4878-b029-a368b8834b35.xml   
73   be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
74   c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
75   c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
76   c1417907-53ee-4d31-a753-cd008516c324.xml   
77   c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
78   c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
79   c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
80   c512c537-32fb-46b6-adaa-4589799569bc.xml   
81   cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
82   ce1115df-1248-419f-a246-e1f6676556bd.xml   
83   d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
84   d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
85   da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
86   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
87   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
88   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
89   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
90   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
91   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
92   da32c93e-b332-4e10-b07c-6be7035d575b.xml   
93   dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
94   e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
95   e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
96   ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
97   f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
98   f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
99   fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
100  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                 Title  \
0    Cancer immunotherapy from biology to nanomedicine   
1    Safety and efficacy of tolebrutinib, an oral b...   
2    Safety and efficacy of tolebrutinib, an oral b...   
3    Safety and efficacy of tolebrutinib, an oral b...   
4    Safety and efficacy of tolebrutinib, an oral b...   
5    Safety and efficacy of tolebrutinib, an oral b...   
6    Safety and efficacy of tolebrutinib, an oral b...   
7    Safety and efficacy of tolebrutinib, an oral b...   
8    Approvals of drugs with uncertain benefit–risk...   
9    Reducing clinical trial risk in multiple scler...   
10   Monoclonal antibodies as immunomodulatory ther...   
11   Mechanisms of action of therapeutic antibodies...   
12   Update in immunosuppressive therapy of myasthe...   
13   Anti-CD20–mediated B-cell depletion in autoimm...   
14   Therapeutic strategies targeting B-cells in mu...   
15   Trial of intrathecal rituximab in progressive ...   
16     B cell tar

## other


In [715]:
#extraction for NLM

import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree

RootDir1 = r'Downloads/HAWK AI/Tasks/NLM_missing_doi_folder'


#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "title"
section_paratag = "p"
bodycount = 0 
full_body_text = []
body_list = []
# title_list = []
doi_list = []
url_list = []
doicount = 0
urlcount = 0
titlecount = 0

nlm_body_list = []
nlm_bodycount = 0
full_body_text_nlm = []
nlm_abstract_list = []
nlm_abstract_count = 0
full_abstract_text = []


tagname1 = 'body' 
tagname2 = 'abstract' 
tagname3 = 'ext-link'
tagname4 = 'ext-link'
tagname5 = 'article-title'
tagname6 = 'article-id'
tagname7  = 'pub-id-type'
file_title = []
nlmcount = 0
doiiter = 0
# elements = set()
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)       
        with open(SourceFolder) as name:
            try:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    nlmcount = nlmcount +1
                    tree = etree.parse(SourceFolder)           
                    roots = tree.getroot()           
                    
                     #get title
#                     for tag in roots.iter(tagname5):
#                         title_list.append(str(tag.text))
                    
#                         titlecount = titlecount + 1

                    title_list = []
                    for elem in roots.iter():
        #                 elements.add(elem.tag)
                        if elem.tag == tagname5:
                            title_list.append(elem.text)

                    file_title.append(title_list[0])
#                     print(title_list[0])

                    #get doi  
                    
                    for tag in roots.iter(tagname6):
                        try:
                            if (tag.attrib) == {'pub-id-type': 'doi'}:
                                doiiter = doiiter + 1
                                doi_list.append(str(tag.text))
                                doicount = doicount + 1
                            
                                
                        except:
                            print("error")
#                             print(tag.text)
#                             doi_list.append("None")
#                             doicount = doicount + 1
                     
                    #get url
                    for tag in roots.iter(tagname6):
                        try:
                            if (tag.attrib) == {'pub-id-type': 'doi'}:
#                                 print(tag.text)
                                url_list.append(str(tag.text))
                                urlcount = urlcount + 1
                        except:
                            url_list.append("None")
                            urlcount = urlcount + 1
                

                   
            except:
                print("error in parsing")



# print(nlmcount)
# print(len(file_title))
# print(len(doi_list))
# print(len(url_list))


df_interim = pd.DataFrame(list(zip(file_title, doi_list, url_list)),
               columns =['Title','DOI','URL'])



df_interim.shape


error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing
error in parsing


(187, 3)

In [716]:
df_interim

Title  \
0    Successful Treatment of Relapsed/Refractory Ex...   
1    Monoclonal Antibodies: Leading Actors in the R...   
2    Response Prediction and Evaluation Using PET i...   
3    Optical Coherence Tomography and Visual Evoked...   
4    Molecular insights into pathogenesis and targe...   
5    Monoclonal Antibodies for the Treatment of Mul...   
6    Patterns and Predictors of First-Line Taxane U...   
7    Magic year for multiple myeloma therapeutics: ...   
8    Exploiting Radiation Therapy to Restore Immune...   
9    Keeping Myeloma in Check: The Past, Present an...   
10   Oncolytic Virotherapy Treatment of Breast Canc...   
11   Development of humoral and cellular immunologi...   
12   Cost-effectiveness of Nivolumab-Ipilimumab Com...   
13   Refractory IgD Multiple Myeloma Treated with D...   
14   Protein Expression in Metastatic Melanoma and ...   
15     Nucleic Acid-Based Approaches for Tumor Therapy   
16   Diagnosis and Molecular Profiles of Large Cell...   
17   Cancer Immunotherapy Dosing: A Pharmacokinetic...   
18   Molecular Immunotherapy: Promising Approach to...   
19   Meta-Analysis of Isolated Hepatic Perfusion an...   
20   Synergistic Effects of Venetoclax and Daratumu...   
21   A vision of immuno-oncology: the Siena think t...   
22   Modulation of Gut Microbiota to Enhance Effect...   
23   Immune checkpoint inhibitors as a real hope in...   
24   Novel Combination Therapies for the Treatment ...   
25   A Case Series of Metastatic Metaplastic Breast...   
26   Therapeutic Potential of Innate Lymphoid Cells...   
27   Immunotherapy Treatment for Triple Negative Br...   
28    Advances in Immunotherapy for Adult Glioblastoma   
29   A Contemporary Review of Immune Checkpoint Inh...   
..                                                 ...   
157  Balancing the CD38 Expression on Effector and ...   
158  Mcl-1 Inhibition: Managing Malignancy in Multi...   
159  Therapeutic Updates for Relapsed and Refractor...   
160  Novel Non-Immunologic Agents for Relapsed and ...   
161  Elucidation of Novel Molecular Targets for The...   
162  A Durable Response With the Combination of Niv...   
163                         α-Radioimmunotherapy with    
164  Targeting the PD-1 Axis with Pembrolizumab for...   
165  Therapeutic Advancements Across Clinical Stage...   
166  Mismatch repair deficiency/microsatellite inst...   
167  Pembrolizumab in advanced osteosarcoma: result...   
168  Harnessing the Immune System to Fight Multiple...   
169  FK228 sensitizes radioresistant small cell lun...   
170  TILs Immunophenotype in Breast Cancer Predicts...   
171  Circulating Tumor DNA as a Prognostic Determin...   
172  Comparisons of Underlying Mechanisms, Clinical...   
173              ANCA-Associated Vasculitis: An Update   
174  Model-Based Meta-Analysis in Psoriasis: A Quan...   
175  Novel and Emerging Therapies for Inflammatory ...   
176  Are tumor size changes predictive of survival ...   
177  Carfilzomib with immunomodulatory drugs for th...   
178  Precision medicine for adjuvant chemotherapy o...   
179  Monoclonal antibodies in relapsed/refractory m...   
180  Hypomethylating Agents and Immunotherapy: Ther...   
181  Altered Immunoregulation in Rheumatoid Arthrit...   
182  The use of single armed observational data to ...   
183  The Crosstalk Between Tumor Cells and the Immu...   
184  Advances in the Treatment of Relapsed and Refr...   
185  Immunotherapy in Adrenocortical Carcinoma: Pre...   
186  Highlights from ASCO-GI 2021 from EORTC Gastro...   

                                   DOI                                URL  
0             10.3389/fmed.2021.649824           10.3389/fmed.2021.649824  
1                   10.3390/ph13120426                 10.3390/ph13120426  
2              10.3390/cancers13123083            10.3390/cancers13123083  
3            10.3389/fnins.2021.692599          10.3389/fnins.2021.692599  
4           10.1186/s40164-020-00188-w         10.1186/s40164-020-00188-

In [8]:
RootDir1 = r'C:/Users/GOWRICH1/Downloads/HAWK AI/Tasks/NLM_missing_doi_folder'
import xml.etree.ElementTree as etree
import pandas as pd


xml_title_list=[]
actual_title_list=[]
actual_journal_list=[]
actual_doi_list=[]
actual_pmc_list=[]
actual_pmid_list=[]

for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)
        #print(SourceFolder)
        xml_name=name
        with open(SourceFolder,'rb') as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                #print(SourceFolder)
                tree = etree.parse(SourceFolder)
                roots = tree.getroot()
                
                
                for tag in roots.iter('front'):
                    tagtext = ElementTree.tostring(tag) 
                    #print(tagtext)
                    tagtext= (str(tagtext, 'UTF-8'))
                    cleanr = re.compile(r'(<(?!/article-title|article-title|front|/front|journal-title|/journal-title|article-id|/article-id).*?>)|(\n)')
                    tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                    #print(tagtext)
                    root_1 = ElementTree.fromstring(tagtext)               
                    title_sen=""
                    journal_sen=""
                    doi_sen=""
                    pmc_sen=""
                    pmid_sen=""
                    for elem in root_1.iter():
                        #print(elem)
                        if elem.tag == "article-title":                    
                            title_sen = title_sen+elem.text
                        if elem.tag =="journal-title":
                            journal_sen= journal_sen+elem.text
                        if elem.attrib == {'pub-id-type':'doi'}:
                            doi_sen= doi_sen+elem.text
                        if elem.attrib == {'pub-id-type':'pmc'}:
                            pmc_sen= pmc_sen+elem.text    
                        if elem.attrib == {'pub-id-type':'pmid'}:
                            pmid_sen= pmid_sen+elem.text     
                            
                    actual_journal_list.append(journal_sen)
                    actual_title_list.append(title_sen)
                    actual_doi_list.append(doi_sen)
                    xml_title_list.append(xml_name)
                    actual_pmc_list.append(pmc_sen)
                    actual_pmid_list.append(pmid_sen)

title_df_NLM = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list,actual_doi_list,actual_pmc_list,actual_pmid_list)),
               columns =['XML','Title','Journal Title','DOI','PMC Number','PM Id'])

In [9]:
title_df_NLM

XML  \
0    0238b48c-9dec-4286-81c6-8c6e79f17c5a.xml   
1    02f98121-708e-48d1-8e4c-518e9f985a48.xml   
2    03fdbd7f-c227-4a17-b423-d3bf2962b53c.xml   
3    0406a3aa-efb3-4371-a044-477b2edcafeb.xml   
4    042ac234-4b7a-493e-ab9d-a708ac6755b5.xml   
5    05644176-4eab-4e49-a492-bec1f74f9402.xml   
6    0687b9eb-49a3-41b7-8fe5-6b312600981a.xml   
7    084d6200-3513-4152-becf-4c58d09f634a.xml   
8    08972e8b-9867-4308-9480-39033bca6668.xml   
9    08a3d9bb-fce8-444f-b377-f27c502a49fd.xml   
10   0a466407-16c7-4568-8a6b-e5798de0d441.xml   
11   0aadf051-3352-4a49-8744-631de0697f0a.xml   
12   0ae59811-703f-455e-9f09-f6a424ce07be.xml   
13   0baab717-67f7-4515-aede-fdc4de4ee9d8.xml   
14   0d15a70a-ce5c-480e-8a88-5f310b29276c.xml   
15   0dae4c12-b1d9-4e2f-a651-df4e5556c575.xml   
16   0ed3d18c-88f8-40b4-8cd3-c3334acfdc1e.xml   
17   0f03d4bb-741e-48f6-a93c-58dc56026894.xml   
18   0fb372d7-59d9-4d32-85f5-47f9698d25a5.xml   
19   103c7f16-6d56-40bb-97cb-6514b68ced3a.xml   
20   116e6565-04c0-4423-921b-844fe948eccb.xml   
21   1260059c-f525-40a8-aacb-d016664ab183.xml   
22   13407424-d984-45d2-8d66-c220aa1c44fb.xml   
23   1641669d-f791-400c-8529-c226f075217f.xml   
24   166875d9-7890-43af-87c5-7e984260e716.xml   
25   16c54d6b-9939-443c-aade-d85406ed92c2.xml   
26   17d35e38-9813-49ed-9a62-97b131ac14f8.xml   
27   1a7149c9-fb12-4b9e-a9fe-09fadc53f0c0.xml   
28   1bd00a4c-dc13-44cc-b439-52aa1a2b35c9.xml   
29   1c052979-2d9c-46bb-94bb-7311d71280b0.xml   
..                                        ...   
162  d1a11ffd-1696-40ae-96b2-335df6751a4a.xml   
163  d65be413-8568-407b-ad11-b229b1a2d814.xml   
164  db65f811-bf80-4895-9f42-a649511ce562.xml   
165  dd31420a-b88b-49fc-8dcc-aac6908163be.xml   
166  de5a4a21-0a68-4d13-bc9e-02bc02343888.xml   
167  e0228a37-a365-419f-95de-c338f490c3ba.xml   
168  e05d9575-4c56-4b27-b33d-d373a83e3c5f.xml   
169  e1039778-6a45-4c39-8e6e-4cacefc73d0c.xml   
170  e10a37ae-4284-46ef-8754-9b3fbcca342a.xml   
171  e11920e3-ed06-4f41-acb4-e0b0b778e35e.xml   
172  e56d3e4f-6dd8-4741-88d8-5a66c48cbf4b.xml   
173  e57a1c9c-049e-40db-9846-d02c74039eff.xml   
174  e585076e-f834-4e1d-93d3-356d4bb21c38.xml   
175  e6a48079-7e3d-4e6b-95ee-9e68f0db79db.xml   
176  e7fa1ca8-2606-4399-982c-985b18ff5fba.xml   
177  e96f6107-49bc-45a5-b9dc-a7937ce25656.xml   
178  ee767037-9381-483c-bda8-19cbecdc2551.xml   
179  ee887a16-9e6d-4a91-952d-5bf018ea66f2.xml   
180  ef046fac-8a69-4bcc-aa2d-a8df2cf448a7.xml   
181  f133ae43-30ea-410c-b6f9-82651e4a027d.xml   
182  f35ccf50-9f19-4a58-847d-f82b56b0ac4d.xml   
183  f3c099f0-3fa2-4284-9f83-bcf7979f1d2e.xml   
184  f535305c-72dc-4eaa-938f-4005480a98c5.xml   
185  f5fac594-040e-42bc-8e81-220bb9907ced.xml   
186  f91629cb-dd77-459f-b9a1-edc75705ea1d.xml   
187  f9c2b530-8a5c-4762-b5ce-c2364311f243.xml   
188  fa9e9312-ba6a-4f9f-a74e-3b3370768664.xml   
189  fb662757-43ad-4de0-889f-2729f2745c4f.xml   
190  fda1d15a-8d21-4d9e-81e5-8b4dfd1633f0.xml   
191  fe160017-5012-4254-ac63-66f0c1dd6e59.xml   

                                                 Title  \
0    Successful Treatment of Relapsed/Refractory Ex...   
1    Monoclonal Antibodies: Leading Actors in the R...   
2    Response Prediction and Evaluation Using PET i...   
3    Optical Coherence Tomography and Visual Evoked...   
4    Molecular insights into pathogenesis and targe...   
5    Monoclonal Antibodies for the Treatment of Mul...   
6    Patterns and Predictors of First-Line Taxane U...   
7    Magic year for multiple myeloma therapeutics: ...   
8    Exploiting Radiation Therapy to Restore Immune...   
9    Keeping Myeloma in Check: The Past, Present an...   
10   Oncolytic Virotherapy Treatment of Breast Canc...   
11   Development of humoral and cellular immunologi...   
12   Cost-effectiveness of Nivolumab-Ipilimumab Com...   
13   Refractory IgD Multiple Myeloma Treated with D...   
14   Protein Expression in Metastatic Melanoma and ...   
15     Nucleic Acid-Based Approaches for Tumor Therapy   
16   Diagnosis an

In [732]:
writer = pd.ExcelWriter('C:/Users/GOWRICH1/Downloads/NLM_Extraction.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
title_df_NLM.to_excel(writer)
writer.close()
